In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\TripLeg_to_Trip.xlsx')

In [5]:
len(df)

238883

###
🧮 Yolculuk Verisinin Gruplandırılması ve Özelleştirilmiş Özetin Çıkarılması

Bu çalışmada, yolculuk verileri `same_p_and_k_y` kolonuna göre gruplanmakta ve belirli kurallar çerçevesinde özetlenmektedir:#### 📌 Kullanılan Kurallar:

- **En küçük `id`'ye sahip satırdan** alınan kolonlar:
  - `start_time`, `orig_dist`, `orig_loc_type`, `purpose`

- **En büyük `id`'ye sahip satırdan** alınan kolonlar:
  - `end_time`, `dest_lat`, `dest_lon`, `dest_dist`, `dest_loc_type`

- **Toplam alınan kolonlar (gruba ait tüm satırlar üzerinden)**:
  - `travel_time`, `vehicle_time`, `waiting_time`, `egress_time`, `as_time`

#### ⚙️ İşlem Adımları:

1. `df` DataFrame’i `id` kolonuna göre sıralanır.
2. `groupby().first()` ile her grup için en küçük id’ye sahip satır alınır.
3. `groupby().last()` ile her grup için en büyük id’ye sahip satır alınır.
4. `groupby().sum()` ile süre verileri toplanır.
5. Yeni bir `result` tablosu oluşturularak istenen kolonlar uygun yerlerden alınır.
6. `reset_index()` ile tablo formatı düzenlenir.

Bu yöntemle, her bir `same_p_and_k_y` grubu için bir satırdan oluşan özet yolculuk verisi elde edilir.


In [6]:
import pandas as pd

# id'ye göre sırala
df_sorted = df.sort_values('id')

# En küçük id'li satırlar
first_rows = df_sorted.groupby('same_p_and_k_y').first()

# En büyük id'li satırlar
last_rows = df_sorted.groupby('same_p_and_k_y').last()

# Toplam değerler
travel_time_sum = df_sorted.groupby('same_p_and_k_y')['travel_time'].sum()
vehicle_time_sum = df_sorted.groupby('same_p_and_k_y')['vehicle_time'].sum()
waiting_time_sum = df_sorted.groupby('same_p_and_k_y')['waiting_time'].sum()
egress_time_sum = df_sorted.groupby('same_p_and_k_y')['egress_time'].sum()
access_time_sum = df_sorted.groupby('same_p_and_k_y')['access_time'].sum()

# Yeni dataframe oluştur
result = first_rows.copy()
result['start_time'] = first_rows['start_time']
result['orig_dist'] = first_rows['orig_dist']
result['dest_loc_type'] = last_rows['dest_loc_type']
result['orig_loc_type'] = first_rows['orig_loc_type']
result['purpose'] = first_rows['purpose']
result['end_time'] = last_rows['end_time']
result['dest_lat'] = last_rows['dest_lat']
result['dest_lon'] = last_rows['dest_lon']
result['dest_dist'] = last_rows['dest_dist']
result['travel_time'] = travel_time_sum
result['vehicle_time'] = vehicle_time_sum
result['waiting_time'] = waiting_time_sum
result['egress_time'] = egress_time_sum
result['access_time'] = access_time_sum

# Index sıfırlanır
result = result.reset_index()


In [7]:
result = result.drop(columns=['Unnamed: 0'], errors='ignore')
result = result.drop(columns=['id_int'], errors='ignore')
result = result.drop(columns=['p_id_clean'], errors='ignore')
result = result.drop(columns=['mode'], errors='ignore')
result = result.drop(columns=['k_y_part'], errors='ignore')
result = result.rename(columns={'Employment Group': 'Employment_Group'})


In [8]:
result.head(1)

,same_p_and_k_y,hh_id,id,p_id,acc_p,person_id,t_leg_id,purpose,gender,orig_lat,orig_lon,dest_lat,dest_lon,orig_loc_type,orig_dist,access_time,egress_time,district,mode_category,Employment_Group,start_time,waiting_time,dest_loc_type,dest_dist,end_time,travel_time,vehicle_time,mode_order,mode_choice,travel_time_group
0,1,aJnHBlKAY9fZeP0yDLWL,1000143,"1,000,168",None,aJnHBlKAY9fZeP0yDLWL-K4,K4Y1E1,Çocuk Alma/Bırakma,Kadın,41.02355499° N,28.86218713° E,41.02355499° N,28.86218713° E,EV(SADECE İKAMET EDİLEN EV),GÜNGÖREN,0,0,GÜNGÖREN,Walking,Child Student,12:00,0,EV(SADECE İKAMET EDİLEN EV),GÜNGÖREN,12:15,15,15,1,Walking,10-20


In [9]:
len(result)

233670

<span style="font-size:22pt; color:red;"><b>Figure 81. Net and Gross Mobility Rate by Person Type (İstanbul SUMP Stage II HTS) </b></span>


In [11]:
profile = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Profile.xlsx')
household = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Household.xlsx')
# 'öğrenci' ifadesini içeren satırları bulmak için yardımcı değişken
is_student = profile['edu'].str.contains('öğrenci', case=False, na=False)

# Koşullar (SQL'deki sıralamaya sadık)
conditions = [
    # 1. Full Time
    (profile['work_status'] == 'Çalışıyor') &
    (profile['age'] >= 18) &
    (profile['work_time'] == 'Tam zamanlı') &
    (~is_student),

    # 2. Part Time
    (profile['work_status'] == 'Çalışıyor') &
    (profile['age'] >= 18) &
    (profile['work_time'] == 'Yarı zamanlı') &
    (~is_student),

    # 3. Adult Student
    (profile['age'] >= 18) & is_student,

    # 4. Adult Unemployed
    (profile['work_status'] == 'Çalışmıyor') &
    (profile['age'].between(18, 65)),

    # 5. 65+ Retired
    (profile['work_status'] == 'Çalışmıyor') &
    (profile['age'] >= 65),

    # 6. Child Student
    (profile['age'].between(6, 18)) & is_student,

    # 7. Child
    (profile['age'].between(6, 18)) & (~is_student)
]

# Bu koşullara karşılık gelen etiketler
choices = [
    'Full Time',
    'Part Time',
    'Adult Student',
    'Adult Unemployed',
    '65+ Retired',
    'Child Student',
    'Child'
]

# Koşul/etiket listelerini tek satırda profile_group kolonuna dönüştür
profile['Employment Group'] = np.select(conditions, choices, default='Other')

missing_count = profile['Employment Group'].isna().sum()
print(f"Employment_Group sütununda {missing_count} adet eksik veri bulunmaktadır.")


C:\Users\Paraboly\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Employment_Group sütununda 0 adet eksik veri bulunmaktadır.


In [12]:
# participation_status = 'evet' olanları filtrele
filtered_profile = profile[
    (profile['participation_status'].str.lower() == 'evet') & 
    (profile['age'] > 5)
]

# Employment Group'a göre kişi sayısı hesapla
employment_counts = filtered_profile['Employment Group'].value_counts(dropna=False).reset_index()

# Kolon isimlerini yeniden adlandır
employment_counts.columns = ['Employment Group', 'Kişi Sayısı']

# En alta toplam satırını ekle
employment_counts.loc[len(employment_counts)] = ['TOPLAM', employment_counts['Kişi Sayısı'].sum()]

employment_counts


,Employment Group,Kişi Sayısı
0,Adult Unemployed,56180
1,Full Time,49172
2,Child Student,21191
3,65+ Retired,14916
4,Adult Student,6383
5,Child,1759
6,Part Time,1711
7,TOPLAM,151312


In [13]:
# 1. Grup bazında benzersiz sayıları hesapla
grouped = result.groupby('Employment_Group').agg({
    'person_id': pd.Series.nunique,
    'same_p_and_k_y': pd.Series.nunique
}).reset_index()

# 2. Kolon isimlerini yeniden adlandır
grouped.columns = ['Employment_Group', 'Yolculuk Yapan Kişi Sayısı', 'Yolculuk Sayısı']

# 3. Toplam satırını oluştur
total_row = pd.DataFrame({
    'Employment_Group': ['TOPLAM'],
    'Yolculuk Yapan Kişi Sayısı': [grouped['Yolculuk Yapan Kişi Sayısı'].sum()],
    'Yolculuk Sayısı': [grouped['Yolculuk Sayısı'].sum()]
})

# 4. Toplam satırını en alta ekle
final_df = pd.concat([grouped, total_row], ignore_index=True)
final_df

,Employment_Group,Yolculuk Yapan Kişi Sayısı,Yolculuk Sayısı
0,65+ Retired,7754,16998
1,Adult Student,4964,10697
2,Adult Unemployed,26073,58515
3,Child,541,1191
4,Child Student,19235,42316
5,Full Time,45397,96235
6,Other,2508,5155
7,Part Time,1226,2563
8,TOPLAM,107698,233670


In [14]:
# 1. İsim uyumsuzluklarını gider: kolon isimlerini eşitle
employment_counts.rename(columns={'Employment Group': 'Employment_Group'}, inplace=True)

# 2. Tabloyu Employment_Group üzerinden birleştir
merged = pd.merge(employment_counts, final_df, on='Employment_Group', how='outer')

# 3. Kolon sırasını isteğe göre düzenle
merged = merged[['Employment_Group', 'Kişi Sayısı', 'Yolculuk Yapan Kişi Sayısı', 'Yolculuk Sayısı']]

# 4. NaN değerleri 0 yap (özellikle 'TOPLAM' satırında bazı kolonlar boş olabilir)
merged.fillna(0, inplace=True)

# 5. Sayısal kolonları tam sayıya çevir (isteğe bağlı)
merged[['Kişi Sayısı', 'Yolculuk Yapan Kişi Sayısı', 'Yolculuk Sayısı']] = merged[[
    'Kişi Sayısı', 'Yolculuk Yapan Kişi Sayısı', 'Yolculuk Sayısı'
]].astype(int)

merged


,Employment_Group,Kişi Sayısı,Yolculuk Yapan Kişi Sayısı,Yolculuk Sayısı
0,Adult Unemployed,56180,26073,58515
1,Full Time,49172,45397,96235
2,Child Student,21191,19235,42316
3,65+ Retired,14916,7754,16998
4,Adult Student,6383,4964,10697
5,Child,1759,541,1191
6,Part Time,1711,1226,2563
7,TOPLAM,151312,107698,233670
8,Other,0,2508,5155


In [15]:
# Kopya al, orijinali bozmamak için (opsiyonel)
merged_with_ratios = merged.copy()

# Brüt ve Net yolculuk oranlarını hesapla
merged_with_ratios['Brüt Yolculuk Oranı'] = merged_with_ratios['Yolculuk Sayısı'] / merged_with_ratios['Kişi Sayısı']
merged_with_ratios['Net Yolculuk Oranı'] = merged_with_ratios['Yolculuk Sayısı'] / merged_with_ratios['Yolculuk Yapan Kişi Sayısı']

# Sonsuz ve NaN değerleri sıfırla (örneğin TOPLAM satırında sıfıra bölme olabilir)
merged_with_ratios.replace([float('inf'), float('nan')], 0, inplace=True)

# Oranları virgülden sonra 2 basamakla sınırlamak istersen:
merged_with_ratios['Brüt Yolculuk Oranı'] = merged_with_ratios['Brüt Yolculuk Oranı'].round(2)
merged_with_ratios['Net Yolculuk Oranı'] = merged_with_ratios['Net Yolculuk Oranı'].round(2)
merged_with_ratios


,Employment_Group,Kişi Sayısı,Yolculuk Yapan Kişi Sayısı,Yolculuk Sayısı,Brüt Yolculuk Oranı,Net Yolculuk Oranı
0,Adult Unemployed,56180,26073,58515,1.04,2.24
1,Full Time,49172,45397,96235,1.96,2.12
2,Child Student,21191,19235,42316,2.00,2.20
3,65+ Retired,14916,7754,16998,1.14,2.19
4,Adult Student,6383,4964,10697,1.68,2.15
5,Child,1759,541,1191,0.68,2.20
6,Part Time,1711,1226,2563,1.50,2.09
7,TOPLAM,151312,107698,233670,1.54,2.17
8,Other,0,2508,5155,0.00,2.06


<span style="font-size:22pt; color:red;"><b>TFigure 141. Trip Rate for Male- Figure 142. Trip Rate for Female</b></span>


In [16]:
# 1. Her kişiden sadece bir tane olacak şekilde filtrele
unique_df = result.drop_duplicates(subset='person_id')

# 2. district ve gender bazında kişi sayısı (unique)
gender_counts = (
    unique_df.groupby(['district', 'gender'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 3. district ve gender bazında toplam yolculuk sayısı (tüm satırlar)
trip_counts = (
    result.groupby(['district', 'gender'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Sadece Erkek ve Kadın olanları al
gender_counts = gender_counts[['district'] + [col for col in ['Erkek', 'Kadın'] if col in gender_counts.columns]]
trip_counts = trip_counts[['district'] + [col for col in ['Erkek', 'Kadın'] if col in trip_counts.columns]]

# 5. Kolon adlarını yeniden adlandır
gender_counts = gender_counts.rename(columns={
    'Erkek': 'Yolculuk Yapan Erkek',
    'Kadın': 'Yolculuk Yapan Kadın'
})

trip_counts = trip_counts.rename(columns={
    'Erkek': 'Erkek Yolculuk Sayısı',
    'Kadın': 'Kadın Yolculuk Sayısı'
})

# 6. İki tabloyu birleştir
merged = gender_counts.merge(trip_counts, on='district', how='left')

# 7. Genel toplam satırını ekle
total_row = merged.drop(columns='district').sum(numeric_only=True)
total_row['district'] = 'Toplam'
merged = pd.concat([merged, pd.DataFrame([total_row])], ignore_index=True)

# 8. Sonuç
merged.head(1)


gender,district,Yolculuk Yapan Erkek,Yolculuk Yapan Kadın,Erkek Yolculuk Sayısı,Kadın Yolculuk Sayısı
0,ADALAR,40,39,101,101


In [17]:
import pandas as pd
# participation_status = 'evet' olanları filtrele
filtered_profile_2 = profile[
    (profile['participation_status'].str.lower() == 'evet') &
    (profile['age'] > 5)
]
# İlçe ve cinsiyet bazında sayım
gender_counts_ = (
    filtered_profile_2.groupby(['District', 'gender'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Toplam satırını hesapla
total_row = gender_counts_.sum(numeric_only=True)
total_row['District'] = 'Toplam'

# Toplam satırını dataframe'e ekle
gender_counts_ = pd.concat([gender_counts_, pd.DataFrame([total_row])], ignore_index=True)
gender_counts_ = gender_counts_.rename(columns={
    'Erkek': 'Toplam Erkek Sayısı',
    'Kadın': 'Toplam Kadın Sayısı',
    'District': 'district'
})
gender_counts_.head(1)

gender,district,Belirtmek istemiyor,Toplam Erkek Sayısı,Toplam Kadın Sayısı
0,ADALAR,0,41,41


In [18]:
# Merge işlemi
merged_final = merged.merge(
    gender_counts_[['district', 'Toplam Erkek Sayısı', 'Toplam Kadın Sayısı']],
    on='district',
    how='left'
)

# Oran kolonlarını hesapla (1 hane olacak şekilde)
merged_final['Brüt Erkek Yolculuk Oranı'] = (
    merged_final['Erkek Yolculuk Sayısı'] / merged_final['Toplam Erkek Sayısı']
).round(1)

merged_final['Net Erkek Yolculuk Oranı'] = (
    merged_final['Erkek Yolculuk Sayısı'] / merged_final['Yolculuk Yapan Erkek']
).round(1)

merged_final['Brüt Kadın Yolculuk Oranı'] = (
    merged_final['Kadın Yolculuk Sayısı'] / merged_final['Toplam Kadın Sayısı']
).round(1)

merged_final['Net Kadın Yolculuk Oranı'] = (
    merged_final['Kadın Yolculuk Sayısı'] / merged_final['Yolculuk Yapan Kadın']
).round(1)

# İstenen kolonları seç
merged_final = merged_final[
    ['district','Brüt Erkek Yolculuk Oranı', 'Net Erkek Yolculuk Oranı', 'Brüt Kadın Yolculuk Oranı', 'Net Kadın Yolculuk Oranı']
]

# Sonuç
merged_final

gender,district,Brüt Erkek Yolculuk Oranı,Net Erkek Yolculuk Oranı,Brüt Kadın Yolculuk Oranı,Net Kadın Yolculuk Oranı
0,ADALAR,2.5,2.5,2.5,2.6
1,ARNAVUTKÖY,1.9,2.0,1.3,2.2
2,ATAŞEHİR,1.8,2.1,1.5,2.2
3,AVCILAR,1.7,2.1,1.4,2.2
4,BAHÇELİEVLER,1.6,2.1,1.3,2.2
5,BAKIRKÖY,1.6,2.1,1.4,2.1
6,BAYRAMPAŞA,2.0,2.2,1.7,2.3
7,BAĞCILAR,1.7,2.1,1.2,2.3
8,BAŞAKŞEHİR,1.9,2.1,1.4,2.3
9,BEYKOZ,1.8,2.0,1.5,2.1


<span style="font-size:22pt; color:red;"><b>Table 28. 2006 - 2012 - 2024 Intercontinental Mobility Rates</b></span>

### 

Bu tablo, İstanbul ilçeleri içerisinde gerçekleşen yolculuklarda başlangıç (orig_continent) ve varış (dest_continent) konumlarının kıta bilgilerine göre dağılımını göstermektedir.

In [19]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# İstanbul ilçelerine ait satırları filtrele
result_istanbul = result[result['district'].isin(istanbul_ilceleri)].copy()



continent_mapping = {
    'BEYKOZ': 'Asia', 'ESENLER': 'Europe', 'SARIYER': 'Europe', 'KADIKÖY': 'Asia', 'SULTANGAZİ': 'Europe',
    'GÜNGÖREN': 'Europe', 'BAĞCILAR': 'Europe', 'ARNAVUTKÖY': 'Europe', 'ESENYURT': 'Europe', 'BAKIRKÖY': 'Europe',
    'MALTEPE': 'Asia', 'BEYOĞLU': 'Europe', 'GAZİOSMANPAŞA': 'Europe', 'ŞİŞLİ': 'Europe', 'EYÜPSULTAN': 'Europe',
    'PENDİK': 'Asia', 'ZEYTİNBURNU': 'Europe', 'BAHÇELİEVLER': 'Europe', 'AVCILAR': 'Europe', 'KÜÇÜKÇEKMECE': 'Europe',
    'FATİH': 'Europe', 'ÜSKÜDAR': 'Asia', 'BEYLİKDÜZÜ': 'Europe', 'KAĞITHANE': 'Europe', 'BEŞİKTAŞ': 'Europe',
    'BAYRAMPAŞA': 'Europe', 'BAŞAKŞEHİR': 'Europe', 'TUZLA': 'Asia', 'ÜMRANİYE': 'Asia', 'BÜYÜKÇEKMECE': 'Europe',
    'ADALAR': 'Europe', 'KARTAL': 'Asia', 'ATAŞEHİR': 'Asia', 'ÇATALCA': 'Europe', 'ÇEKMEKÖY': 'Asia',
    'SULTANBEYLİ': 'Asia', 'SANCAKTEPE': 'Asia', 'SİLİVRİ': 'Europe', 'ŞİLE': 'Asia', 'ÇAYIROVA': 'Asia',
    'GEBZE': 'Asia', 'SÜLEYMANPAŞA': 'Europe', 'GÖLCÜK': 'Asia', 'İZMİT': 'Asia', 'DİLOVASI': 'Asia', 'DARICA': 'Asia',
    'ÇORLU': 'Europe', 'DERİNCE': 'Asia', 'ÇERKEZKÖY': 'Europe', 'BAŞİSKELE': 'Asia', 'KÖRFEZ': 'Asia', 'MARMARAEREĞLİSİ': 'Europe',
    'KARTEPE': 'Asia', 'ERGENE': 'Europe', 'ŞARKÖY': 'Europe', 'KAPAKLI': 'Europe', 'KANDIRA': 'Asia', 'SARAY': 'Europe'
}

# Hem 'first_orig_dist' hem de 'first_dest_dist' kolonlarına kıta bilgisi eklemek için map fonksiyonunu kullanıyoruz.
result_istanbul['orig_continent'] = result_istanbul['orig_dist'].map(continent_mapping)
result_istanbul['dest_continent'] = result_istanbul['dest_dist'].map(continent_mapping)

result_istanbul["continental_type"] = result_istanbul.apply(
    lambda row: "Within Europe" if row["orig_continent"] == "Europe" and row["dest_continent"] == "Europe"
    else "Within Asia" if row["orig_continent"] == "Asia" and row["dest_continent"] == "Asia"
    else "Intercontinental",
    axis=1
)

# 1. Sayısal matris (frekans)
continent_matrix = pd.crosstab(
    result_istanbul['orig_continent'],
    result_istanbul['dest_continent']
)

# 2. Genel toplam (tüm hücrelerin toplamı)
total_sum = continent_matrix.values.sum()

# 3. Her hücre için yüzdesel değerleri hesapla
continent_matrix_total_percent = (continent_matrix / total_sum) * 100
continent_matrix_total_percent = continent_matrix_total_percent.round(2)  # yüzde değerlerini yuvarla

# 4. Göstermek için birlikte sun (isteğe bağlı)
combined_total_view = pd.concat([
    continent_matrix,
    continent_matrix_total_percent.add_suffix(" (%)")
], axis=1)

combined_total_view


dest_continent,Asia,Europe,Asia (%),Europe (%)
orig_continent,,,,
Asia,71996,2489,32.47,1.12
Europe,2433,144799,1.10,65.31



<span style="font-size:22pt; color:red;"><b>Figure 82. Travel Purposes </b></span>


Bu tablo, "Eve gidiş" dışındaki yolculukları temsili gruplar altında toplayarak her bir grubun:
- Toplam yolculuk sayısını (`trip_count`),
- Tüm yolculuklar içindeki yüzde payını (`percentage`),

göstermektedir.

#### Gruplar:
- **Work:** İşe gidiş
- **School:** Okula gidiş (ilk-orta-lise-üniversite)
- **Health:** Sağlık kuruluşlarına gidiş
- **Leisure:** Alışveriş, arkadaş ziyareti, spor, yeme-içme vb.
- **Pick-up/Drop-off:** Çocuk ya da diğer bireyleri alma/bırakma
 bulundurulmalıdır.


In [20]:
# 1. "Eve gidiş" dışındaki satırları filtrele
filtered = result[result['purpose'] != 'Eve gidiş'].copy()

# 2. first_purpose değerlerini gruplayarak yeni sütun oluştur
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 3. Yeni kolon ekle
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. NULL olmayanlarla çalış ve grup bazında say
grouped = filtered[filtered['purpose_group'].notna()] \
    .groupby('purpose_group') \
    .size() \
    .reset_index(name='trip_count')

# 5. Yüzde hesapla
total_trips = grouped['trip_count'].sum()
grouped['percentage'] = round(grouped['trip_count'] * 100.0 / total_trips, 2)

# 6. Azalan sıraya göre sırala
grouped = grouped.sort_values(by='trip_count', ascending=False).reset_index(drop=True)
grouped

,purpose_group,trip_count,percentage
0,Work,47063,38.47
1,Leisure,39515,32.30
2,School,23613,19.30
3,Pick-up/Drop-off,7407,6.06
4,Health,4726,3.86


In [21]:
# 1. "Eve gidiş" dışındaki satırları filtrele
filtered = result[result['purpose'] != 'Eve gidiş'].copy()

# 2. Sadece İstanbul ilçelerinde geçen verileri al
istanbul_districts = [
    'ADALAR', 'ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER', 'BAKIRKÖY',
    'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ', 'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE',
    'ÇATALCA', 'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜPSULTAN', 'FATİH', 'GAZİOSMANPAŞA',
    'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK',
    'SANCAKTEPE', 'SARIYER', 'ŞİLE', 'SİLİVRİ', 'ŞİŞLİ', 'SULTANBEYLİ', 'SULTANGAZİ',
    'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU'
]
filtered = filtered[filtered['district'].isin(istanbul_districts)]

# 3. first_purpose değerlerini gruplayarak yeni sütun oluştur
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 4. Yeni kolon ekle
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. NULL olmayanlarla çalış ve grup bazında say
grouped = filtered[filtered['purpose_group'].notna()] \
    .groupby('purpose_group') \
    .size() \
    .reset_index(name='trip_count')

# 6. Yüzde hesapla
total_trips = grouped['trip_count'].sum()
grouped['percentage'] = round(grouped['trip_count'] * 100.0 / total_trips, 2)

# 7. Azalan sıraya göre sırala
grouped = grouped.sort_values(by='trip_count', ascending=False).reset_index(drop=True)

grouped


,purpose_group,trip_count,percentage
0,Work,44763,38.62
1,Leisure,37028,31.94
2,School,22547,19.45
3,Pick-up/Drop-off,7151,6.17
4,Health,4425,3.82


In [22]:
tekirdag_districts = [
    'ÇERKEZKÖY', 'ÇORLU', 'ERGENE', 'KAPAKLI', 'MARMARAEREĞLİSİ', 'SARAY'
]

# 1. "Eve gidiş" dışındaki satırları filtrele
filtered = result[result['purpose'] != 'Eve gidiş'].copy()

# 2. Sadece Tekirdağ ilçelerini filtrele
filtered = filtered[filtered['district'].isin(tekirdag_districts)]

# 3. first_purpose değerlerini gruplayarak yeni sütun oluştur
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 4. Yeni kolon ekle
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. NULL olmayanlarla çalış ve grup bazında say
grouped = (
    filtered[filtered['purpose_group'].notna()]
    .groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
)

# 6. Yüzde hesapla
total_trips = grouped['trip_count'].sum()
grouped['percentage'] = round(grouped['trip_count'] * 100.0 / total_trips, 2)

# 7. Azalan sıraya göre sırala
grouped = grouped.sort_values(by='trip_count', ascending=False).reset_index(drop=True)

grouped


,purpose_group,trip_count,percentage
0,Work,1137,38.93
1,Leisure,1026,35.12
2,School,492,16.84
3,Health,190,6.50
4,Pick-up/Drop-off,76,2.60


In [23]:
kocaeli_districts = ['ÇAYIROVA', 'DARICA', 'GEBZE']

# 1. "Eve gidiş" dışındaki satırları filtrele
filtered = result[result['purpose'] != 'Eve gidiş'].copy()

# 2. Sadece Kocaeli ilçelerini filtrele
filtered = filtered[filtered['district'].isin(kocaeli_districts)]

# 3. first_purpose değerlerini gruplayarak yeni sütun oluştur
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 4. Yeni kolon ekle
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. NULL olmayanlarla çalış ve grup bazında say
grouped = (
    filtered[filtered['purpose_group'].notna()]
    .groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
)

# 6. Yüzde hesapla
total_trips = grouped['trip_count'].sum()
grouped['percentage'] = round(grouped['trip_count'] * 100.0 / total_trips, 2)

# 7. Azalan sıraya göre sırala
grouped = grouped.sort_values(by='trip_count', ascending=False).reset_index(drop=True)

grouped


,purpose_group,trip_count,percentage
0,Leisure,1461,41.87
1,Work,1163,33.33
2,School,574,16.45
3,Pick-up/Drop-off,180,5.16
4,Health,111,3.18


In [24]:
# 1. "Eve gidiş" dışındaki satırları filtrele
test = result[result['purpose'] != 'Eve gidiş'].copy()
test

,same_p_and_k_y,hh_id,id,p_id,acc_p,person_id,t_leg_id,purpose,gender,orig_lat,orig_lon,dest_lat,dest_lon,orig_loc_type,orig_dist,access_time,egress_time,district,mode_category,Employment_Group,start_time,waiting_time,dest_loc_type,dest_dist,end_time,travel_time,vehicle_time,mode_order,mode_choice,travel_time_group
0,1,aJnHBlKAY9fZeP0yDLWL,1000143,"1,000,168",None,aJnHBlKAY9fZeP0yDLWL-K4,K4Y1E1,Çocuk Alma/Bırakma,Kadın,41.02355499° N,28.86218713° E,41.02355499° N,28.86218713° E,EV(SADECE İKAMET EDİLEN EV),GÜNGÖREN,0,0,GÜNGÖREN,Walking,Child Student,12:00,0,EV(SADECE İKAMET EDİLEN EV),GÜNGÖREN,12:15,15,15,1,Walking,10-20
2,3,Ujr56uZoxVoVxykDMiMp,1000159,"1,000,182",None,Ujr56uZoxVoVxykDMiMp-K5,K5Y1E1,İşe gidiş,Erkek,41.05813263° N,28.87076477° E,41.05679030° N,28.86342230° E,EV(SADECE İKAMET EDİLEN EV),ESENLER,1,1,ESENLER,Private Vehicle,Full Time,08:00,0,İŞ YERİ,ESENLER,08:10,10,8,1,Private Vehicle,10-20
3,4,fJLpbrIOV1IhJ9M5Gqhn,1000210,"1,000,249",None,fJLpbrIOV1IhJ9M5Gqhn-K2,K2Y1E1,Alışveriş,Kadın,40.99653385° N,28.70731253° E,41.00418310° N,28.68781470° E,EV(SADECE İKAMET EDİLEN EV),AVCILAR,5,3,AVCILAR,Public Transport,Adult Student,20:00,2,DİĞER,ESENYURT,20:20,20,10,1,Public Transport,20-30
4,5,fJLpbrIOV1IhJ9M5Gqhn,1000211,"1,000,250",None,fJLpbrIOV1IhJ9M5Gqhn-K3,K3Y1E1,Alışveriş,Kadın,40.99653385° N,28.70731253° E,41.00418310° N,28.68781470° E,EV(SADECE İKAMET EDİLEN EV),AVCILAR,5,3,AVCILAR,Public Transport,Adult Student,20:00,2,DİĞER,ESENYURT,20:20,20,10,1,Public Transport,20-30
5,6,Tgb3aA0OiDTV9WuNtCMo,1000229,"1,000,288",None,Tgb3aA0OiDTV9WuNtCMo-K3,K3Y1E1,Çocuk Alma/Bırakma,Erkek,41.00512859° N,28.93566613° E,41.00359490° N,28.93542980° E,EV(SADECE İKAMET EDİLEN EV),FATİH,0,0,FATİH,Walking,Child Student,13:00,0,DİĞER,FATİH,13:10,10,10,1,Walking,10-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233660,235530,UtoPDyX2FjVaiUiYG8Bg,117702,"99,999",Tek,UtoPDyX2FjVaiUiYG8Bg-K1,K1Y1E1,Okula gidiş(üniversite ve üzeri),Kadın,41.05796622° N,28.90359000° E,41.09792490° N,28.97965480° E,EV(SADECE İKAMET EDİLEN EV),BAYRAMPAŞA,5,7,BAYRAMPAŞA,Public Transport,Adult Student,08:00,13,OKUL,KAĞITHANE,09:40,100,75,1,Public Transport,30-40
233662,235532,wHviwRiGbzI64belZPwX,735,992,K4,wHviwRiGbzI64belZPwX-K3,K3Y1E1,Okula gidiş(ilk ve ortaöğretim),Kadın,41.06200861° N,28.90155869° E,41.06326280° N,28.89613770° E,EV(SADECE İKAMET EDİLEN EV),BAYRAMPAŞA,0,0,BAYRAMPAŞA,Walking,Child Student,08:00,0,OKUL,BAYRAMPAŞA,08:10,10,10,1,Walking,10-20
233664,235534,PjRQzpA9MlTR88JzqJOP,737,996,"K3, K4",PjRQzpA9MlTR88JzqJOP-K1,K1Y1E1,Çocuk Alma/Bırakma,Kadın,41.02774672° N,28.86644616° E,41.02840790° N,28.86270240° E,EV(SADECE İKAMET EDİLEN EV),GÜNGÖREN,0,0,GÜNGÖREN,Walking,Adult Unemployed,14:00,0,DİĞER,GÜNGÖREN,14:15,15,15,1,Walking,10-20
233666,235536,PjRQzpA9MlTR88JzqJOP,739,996,Tek,PjRQzpA9MlTR88JzqJOP-K1,K1Y3E1,Çocuk Alma/Bırakma,Kadın,41.02774672° N,28.86644616° E,41.02840790° N,28.86270240° E,EV(SADECE İKAMET EDİLEN EV),GÜNGÖREN,0,0,GÜNGÖREN,Walking,Adult Unemployed,18:00,0,DİĞER,GÜNGÖREN,18:30,30,30,1,Walking,30-40


<span style="font-size:22pt; color:red;"><b>Figure 83. Detailed Travel Purpose (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, "Eve gidiş" dışındaki yolculukları temsili gruplar altında toplayarak her bir grubun:
- Toplam yolculuk sayısını (`trip_count`),
- Tüm yolculuklar içindeki yüzde payını (`percentage`),
göstermektedia/bırakma 

In [25]:
# "Eve gidiş" dışındaki değerleri filtrele
filtered = result[result['purpose'] != 'Eve gidiş']

# Sayısal ve yüzdesel dağılımı hesapla
purpose_counts = filtered['purpose'].value_counts()
purpose_percent = filtered['purpose'].value_counts(normalize=True) * 100

# Yeni DataFrame oluştur
purpose_distribution = pd.DataFrame({
    'Count': purpose_counts,
    'Percentage': purpose_percent.round(2)
})

purpose_distribution


,Count,Percentage
purpose,,
İşe gidiş,47063,38.47
Alışveriş,20865,17.06
Okula gidiş(ilk ve ortaöğretim),14368,11.75
Çocuk Alma/Bırakma,7057,5.77
Okula gidiş(lise),6058,4.95
Akraba/Arkadaş Ziyareti,5776,4.72
"Spor, Eğlence vb. gidiş",5705,4.66
Sağlık Kurumuna gidiş,4726,3.86
Diğer,4229,3.46



<span style="font-size:22pt; color:red;"><b>Figure 84. Departure Time By Modes</b></span>

#### 
 **Tablo Açıklaması**:
- `hour_group`: Yolculuğun başladığı saat dilimini belirtir (örneğin: 08:00).
- Diğer sütunlar (ör. `Walking`, `Private Vehicle`, `Public Transport`, `Micromobility`, `Service`): O saat diliminde ilgili ulaşım modunun kaç kez tercih edildiğini gösterir.
- **"Toplam"** satırı ise tüm gün boyunca her ulaşım modunun toplam kullanım sayısını verir.



In [26]:
import pandas as pd

# 1. mode_choice NULL olmayanları filtrele
filtered = result[result['mode_choice'].notna()].copy()

# 2. start_time'ı saatlik gruba çevir (ör. '08:00')
filtered['hour_group'] = pd.to_datetime(filtered['start_time'], errors='coerce').dt.strftime('%H:00')

# 3. hour_group ve mode_choice bazlı sayım (pivot tablo)
hourly_counts = pd.pivot_table(
    filtered,
    index='hour_group',
    columns='mode_choice',
    aggfunc='size',
    fill_value=0
).reset_index()

# 4. Saat sırasına göre sırala
hourly_counts = hourly_counts.sort_values('hour_group').reset_index(drop=True)

# 5. Yüzde hesapla (her sütunun toplamına göre normalize et)
percentage_by_column = hourly_counts.copy()
percentage_only = percentage_by_column.drop(columns='hour_group')

# Her sütunun toplamına göre yüzde hesapla
column_totals = percentage_only.sum(axis=0)
percentage_only = percentage_only.div(column_totals, axis=1) * 100

# hour_group kolonunu geri ekle
percentage_by_column = pd.concat([hourly_counts[['hour_group']], percentage_only], axis=1)

# 6. (İsteğe bağlı) Toplam satırı ekle (yüzde değil, adet için)
total_row = hourly_counts.drop(columns='hour_group').sum(numeric_only=True)
total_row['hour_group'] = 'Toplam'
hourly_counts_with_total = pd.concat([hourly_counts, pd.DataFrame([total_row])], ignore_index=True)

# Çıktılar:
# Sayılar (adet bazlı)
print("Saatlik mode_choice sayıları:")
display(hourly_counts_with_total)

# Sütun bazında yüzdeler
print("Her mode_choice'ın saatlik yüzde dağılımı (%):")
display(percentage_by_column)


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_5020\3091769344.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered['hour_group'] = pd.to_datetime(filtered['start_time'], errors='coerce').dt.strftime('%H:00')


Saatlik mode_choice sayıları:


mode_choice,hour_group,Micromobility,Private Vehicle,Public Transport,Service,Walking
0,00:00,0,11,3,4,12
1,01:00,0,22,14,2,57
2,02:00,0,14,6,3,45
3,03:00,0,19,5,7,18
4,04:00,0,45,11,23,35
5,05:00,1,179,169,221,117
6,06:00,51,1245,1585,1595,882
7,07:00,106,5929,7222,4452,11139
8,08:00,81,7305,5990,2642,13465
9,09:00,25,2328,2105,260,4113


Her mode_choice'ın saatlik yüzde dağılımı (%):


mode_choice,hour_group,Micromobility,Private Vehicle,Public Transport,Service,Walking
0,00:00,0.000000,0.025247,0.006548,0.020924,0.009648
1,01:00,0.000000,0.050493,0.030557,0.010462,0.045828
2,02:00,0.000000,0.032132,0.013096,0.015693,0.036180
3,03:00,0.000000,0.043608,0.010913,0.036617,0.014472
4,04:00,0.000000,0.103282,0.024009,0.120312,0.028140
5,05:00,0.126582,0.410833,0.368867,1.156039,0.094069
6,06:00,6.455696,2.857471,3.459490,8.343359,0.709134
7,07:00,13.417722,13.607987,15.763052,23.288173,8.955836
8,08:00,10.253165,16.766123,13.074035,13.820160,10.825957
9,09:00,3.164557,5.343126,4.594465,1.360046,3.306881


In [27]:
output_path = 'mode_choice_yuzdeler.xlsx'
percentage_by_column.to_excel(output_path, index=False)


<span style="font-size:22pt; color:red;"><b>Figure 92. Mode Choice </b></span>

Bu tablo, `mode_choice` değişkenine göre her bir ulaşım modunun kaç kez tercih edildiğini ve toplam içindeki yüzde (%) oranını göstermektedir.

#### Tablo Açıklaması:
- **mode_choice**: Kullanıcının seçtiği ulaşım modu (örneğin: Walking, Public Transport, Private Vehicle vb.).
- **count**: İlgili ulaşım modunun kaç defa kullanıldığını gösterir.
- **percentage**: Bu modun toplam yolculuklar içindeki yüzdesel oranını belirtir (%, yuvarlatılmış).

In [28]:
import pandas as pd

# 1. mode_category boş olmayanları filtrele
filtered = result[result['mode_choice'].notna()]

# 2. mode_category'ye göre grupla ve say
mode_counts = filtered['mode_choice'].value_counts().reset_index()
mode_counts.columns = ['mode_choice', 'count']

# 3. Yüzde hesapla
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)
mode_counts

,mode_choice,count,percentage
0,Walking,124377,53.23
1,Public Transport,45816,19.61
2,Private Vehicle,43570,18.65
3,Service,19117,8.18
4,Micromobility,790,0.34



<span style="font-size:22pt; color:red;"><b>Figure 106.Change in transportation mode choice for rubber- tired vehicles from 2006 to 2024 </b></span>

Bu tablo, `mode_choice` değişkenine göre her bir ulaşım modunun kaç kez tercih edildiğini ve toplam içindeki yüzde (%) oranını göstermektedir.

#### Tablo Açıklaması:
- **mode_choice**: Kullanıcının seçtiği ulaşım modu (örneğin: Walking, Public Transport, Private Vehicle vb.).
- **count**: İlgili ulaşım modunun kaç defa kullanıldığını gösterir.
- **percentage**: Bu modun toplam yolculuklar içindeki yüzdesel oranını belirtir (%, yuvarlatılmış).
- Yürüme ve mikromobilite dahil edilmememiştir.

In [29]:
# Büyük harfli ilçelere göre filtrele
istanbul_districts = [
    'ADALAR', 'ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER', 'BAKIRKÖY',
    'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ', 'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE',
    'ÇATALCA', 'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜPSULTAN', 'FATİH', 'GAZİOSMANPAŞA',
    'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK',
    'SANCAKTEPE', 'SARIYER', 'ŞİLE', 'SİLİVRİ', 'ŞİŞLİ', 'SULTANBEYLİ', 'SULTANGAZİ',
    'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU'
]

# Filtreleme
istanbul_data = result[
    result['district'].isin(istanbul_districts) & 
    result['mode_choice'].notna()
]

# Mod sayımları
mode_counts = istanbul_data['mode_choice'].value_counts().reset_index()
mode_counts.columns = ['mode_choice', 'count']

# Yüzde hesabı: Walking ve Micromobility hariç toplam üzerinden
exclude_modes = ['Walking', 'Micromobility']
included_total = mode_counts[~mode_counts['mode_choice'].isin(exclude_modes)]['count'].sum()

mode_counts['percentage'] = mode_counts.apply(
    lambda row: round((row['count'] / included_total) * 100, 1)
    if row['mode_choice'] not in exclude_modes else None,
    axis=1
)

mode_counts


,mode_choice,count,percentage
0,Walking,117514,NaN
1,Public Transport,44414,42.9
2,Private Vehicle,41994,40.6
3,Service,17027,16.5
4,Micromobility,768,NaN



<span style="font-size:22pt; color:red;"><b>Figure 93.Changes in mode choice between 2006, 2012 and 2024 (%) (İstanbul SUMP Stage II HTS)  </b></span>

Bu tablo, `mode_choice` değişkenine göre her bir ulaşım modunun kaç kez tercih edildiğini ve toplam içindeki yüzde (%) oranını göstermektedir.

#### Tablo Açıklaması:
- **mode_choice**: Kullanıcının seçtiği ulaşım modu (örneğin: Walking, Public Transport, Private Vehicle vb.).
- **count**: İlgili ulaşım modunun kaç defa kullanıldığını gösterir.
- **percentage**: Bu modun toplam yolculuklar içindeki yüzdesel oranını belirtir (%, yuvarlatılmış).

In [30]:
# Büyük harfli ilçelere göre filtrele (dönüştürme yok!)
istanbul_districts = [
    'ADALAR', 'ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER', 'BAKIRKÖY',
    'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ', 'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE',
    'ÇATALCA', 'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜPSULTAN', 'FATİH', 'GAZİOSMANPAŞA',
    'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK',
    'SANCAKTEPE', 'SARIYER', 'ŞİLE', 'SİLİVRİ', 'ŞİŞLİ', 'SULTANBEYLİ', 'SULTANGAZİ',
    'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU'
]

istanbul_data = result[
    result['district'].isin(istanbul_districts) & 
    result['mode_choice'].notna()
]

# Mode_choice kolonunu say ve yüzdeyi hesapla
mode_counts = istanbul_data['mode_choice'].value_counts().reset_index()
mode_counts.columns = ['mode_choice', 'count']

total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 1)

mode_counts

,mode_choice,count,percentage
0,Walking,117514,53.0
1,Public Transport,44414,20.0
2,Private Vehicle,41994,18.9
3,Service,17027,7.7
4,Micromobility,768,0.3


In [31]:
# Birlikte Yolculuk Dahil değil

result_not_null = result[result["acc_p"].notna()]
result_not_null

# Büyük harfli ilçelere göre filtrele (dönüştürme yok!)
istanbul_districts = [
    'ADALAR', 'ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER', 'BAKIRKÖY',
    'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ', 'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE',
    'ÇATALCA', 'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜPSULTAN', 'FATİH', 'GAZİOSMANPAŞA',
    'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK',
    'SANCAKTEPE', 'SARIYER', 'ŞİLE', 'SİLİVRİ', 'ŞİŞLİ', 'SULTANBEYLİ', 'SULTANGAZİ',
    'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU'
]

istanbul_data = result_not_null[
    result_not_null['district'].isin(istanbul_districts) & 
    result_not_null['mode_choice'].notna()
]

# Mode_choice kolonunu say ve yüzdeyi hesapla
mode_counts = istanbul_data['mode_choice'].value_counts().reset_index()
mode_counts.columns = ['mode_choice', 'count']

total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 1)

mode_counts

,mode_choice,count,percentage
0,Walking,99896,51.3
1,Public Transport,41966,21.5
2,Private Vehicle,36047,18.5
3,Service,16222,8.3
4,Micromobility,756,0.4


In [32]:
# Büyük harfli ilçelere göre filtrele (dönüştürme yok!)
tekirdag_districts = [
   'ÇERKEZKÖY','ÇORLU','ERGENE','KAPAKLI','MARMARAEREĞLİSİ','SARAY'
]

tekirdag_data = result[
    result['district'].isin(tekirdag_districts) & 
    result['mode_choice'].notna()
]

# Mode_choice kolonunu say ve yüzdeyi hesapla
mode_counts = tekirdag_data['mode_choice'].value_counts().reset_index()
mode_counts.columns = ['mode_choice', 'count']

total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)

mode_counts

,mode_choice,count,percentage
0,Walking,3320,58.15
1,Service,854,14.96
2,Private Vehicle,766,13.42
3,Public Transport,759,13.29
4,Micromobility,10,0.18


In [33]:
# Büyük harfli ilçelere göre filtrele (dönüştürme yok!)
kocaeli_districts = [
   'ÇAYIROVA','DARICA','GEBZE'
]

kocaeli_data = result[
    result['district'].isin(kocaeli_districts) & 
    result['mode_choice'].notna()
]

# Mode_choice kolonunu say ve yüzdeyi hesapla
mode_counts = kocaeli_data['mode_choice'].value_counts().reset_index()
mode_counts.columns = ['mode_choice', 'count']

total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)

mode_counts

,mode_choice,count,percentage
0,Walking,3543,56.74
1,Service,1236,19.80
2,Private Vehicle,810,12.97
3,Public Transport,643,10.30
4,Micromobility,12,0.19



<span style="font-size:22pt; color:red;"><b>Figure 94-95-96. Mode Choice in İstanbul According District</b></span>

Bu tablo, her bir ilçede (district) hangi ulaşım modlarının ne sıklıkta kullanıldığını göstermektedir. Veriler, `mode_choice` kolonuna göre gruplanmış ve sadece boş olmayan veriler değerlendirmeye alınmıştır.

#### Tablo Açıklaması:
- **district**: Yolculuğun başladığı ya da hane konumunun bulunduğu ilçe adı.
- **Service**: Servis aracıyla yapılan yolculuk sayısı.
- **Private Vehicle**: Özel araçla yapılan yolculuk sayısı.
- **Public Transport**: Toplu taşıma ile yapılan yolculuk sayısı.
- **Walking**: Yaya olarak yapılan yolculuk sayısı.
- **Micromobility**: Scooter, bisiklet vb. mikro mobilite araçlarıyla yapılan yolculuk sayısı.

In [34]:
# İstanbul ilçeleri listesi
istanbul_districts = [
    'ADALAR', 'ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER', 'BAKIRKÖY',
    'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ', 'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE',
    'ÇATALCA', 'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜPSULTAN', 'FATİH', 'GAZİOSMANPAŞA',
    'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK',
    'SANCAKTEPE', 'SARIYER', 'ŞİLE', 'SİLİVRİ', 'ŞİŞLİ','SULTANBEYLİ', 'SULTANGAZİ',
    'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU'
]

# 1. mode_choice boş olmayanları filtrele
filtered = result[result['mode_choice'].notna()].copy()

# 2. Sadece İstanbul ilçelerini filtrele
filtered = filtered[filtered['district'].isin(istanbul_districts)]

# 3. district ve mode_choice bazında sayım yap (pivot)
grouped = (
    filtered.groupby(['district', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik olabilecek kolonları ekle (sıfırla)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Sadece istenen kolonları sırala
district_mode_counts = grouped[[
    'district', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

district_mode_counts.to_excel("istanbul_choice.xlsx")

In [35]:
# İstanbul ilçeleri listesi
kocaeli_districts = [
   'ÇAYIROVA','DARICA','GEBZE'
]

# 1. mode_choice boş olmayanları filtrele
filtered = result[result['mode_choice'].notna()].copy()

# 2. Sadece İstanbul ilçelerini filtrele
filtered = filtered[filtered['district'].isin(kocaeli_districts)]

# 3. district ve mode_choice bazında sayım yap (pivot)
grouped = (
    filtered.groupby(['district', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik olabilecek kolonları ekle (sıfırla)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Sadece istenen kolonları sırala
district_mode_counts = grouped[[
    'district', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

district_mode_counts.to_excel("kocaeli.xlsx")

In [36]:
# İstanbul ilçeleri listesi
tekirdag_districts = [
   'ÇERKEZKÖY','ÇORLU','ERGENE','KAPAKLI','MARMARAEREĞLİSİ','SARAY'
]

# 1. mode_choice boş olmayanları filtrele
filtered = result[result['mode_choice'].notna()].copy()

# 2. Sadece İstanbul ilçelerini filtrele
filtered = filtered[filtered['district'].isin(tekirdag_districts)]

# 3. district ve mode_choice bazında sayım yap (pivot)
grouped = (
    filtered.groupby(['district', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik olabilecek kolonları ekle (sıfırla)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Sadece istenen kolonları sırala
district_mode_counts = grouped[[
    'district', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

district_mode_counts.to_excel("tekirdağ.xlsx")


<span style="font-size:22pt; color:red;"><b>Figure 102. Distribution of Activity Purposes and Numbers in Intercontinental Trips</b></span>


Bu tablo, sadece `continental_type` değeri **"Intercontinental"** , **"Within Asia"** ve **"Within Europe"** olan yolculuklar dikkate alınarak, yolculukların hangi amaçlarla yapıldığını ve bu grupların toplam içindeki yüzdesel dağılımını göstermektedir.

#### Tablo Açıklaması:
- **purpose_group**: Seyahat amacına göre gruplanmış kategoriler.
  - `Work`: İşe gitme
  - `School`: Okula gitme (ilk-orta-lise-üniversite)
  - `Health`: Sağlık kurumuna gitme
  - `Leisure`: Alışveriş, eğlence, ziyaret vb.
  - `Pick-up/Drop-off`: Çocuk veya başka birini alma/bırakma
  - `Other`: Diğer kategorilere uymayan amaçlar
- **count**: İlgili grubun kaç defa gözlemlendiği
- **percentage**: Bu grubun toplam Intercontinental yolculuklar içindeki oranı (%)


In [37]:
continent_mapping = {
    'BEYKOZ': 'Asia', 'ESENLER': 'Europe', 'SARIYER': 'Europe', 'KADIKÖY': 'Asia', 'SULTANGAZİ': 'Europe',
    'GÜNGÖREN': 'Europe', 'BAĞCILAR': 'Europe', 'ARNAVUTKÖY': 'Europe', 'ESENYURT': 'Europe', 'BAKIRKÖY': 'Europe',
    'MALTEPE': 'Asia', 'BEYOĞLU': 'Europe', 'GAZİOSMANPAŞA': 'Europe', 'ŞİŞLİ': 'Europe', 'EYÜPSULTAN': 'Europe',
    'PENDİK': 'Asia', 'ZEYTİNBURNU': 'Europe', 'BAHÇELİEVLER': 'Europe', 'AVCILAR': 'Europe', 'KÜÇÜKÇEKMECE': 'Europe',
    'FATİH': 'Europe', 'ÜSKÜDAR': 'Asia', 'BEYLİKDÜZÜ': 'Europe', 'KAĞITHANE': 'Europe', 'BEŞİKTAŞ': 'Europe',
    'BAYRAMPAŞA': 'Europe', 'BAŞAKŞEHİR': 'Europe', 'TUZLA': 'Asia', 'ÜMRANİYE': 'Asia', 'BÜYÜKÇEKMECE': 'Europe',
    'ADALAR': 'Europe', 'KARTAL': 'Asia', 'ATAŞEHİR': 'Asia', 'ÇATALCA': 'Europe', 'ÇEKMEKÖY': 'Asia',
    'SULTANBEYLİ': 'Asia', 'SANCAKTEPE': 'Asia', 'SİLİVRİ': 'Europe', 'ŞİLE': 'Asia', 'ÇAYIROVA': 'Asia',
    'GEBZE': 'Asia', 'SÜLEYMANPAŞA': 'Europe', 'GÖLCÜK': 'Asia', 'İZMİT': 'Asia', 'DİLOVASI': 'Asia', 'DARICA': 'Asia',
    'ÇORLU': 'Europe', 'DERİNCE': 'Asia', 'ÇERKEZKÖY': 'Europe', 'BAŞİSKELE': 'Asia', 'KÖRFEZ': 'Asia', 'MARMARAEREĞLİSİ': 'Europe',
    'KARTEPE': 'Asia', 'ERGENE': 'Europe', 'ŞARKÖY': 'Europe', 'KAPAKLI': 'Europe', 'KANDIRA': 'Asia', 'SARAY': 'Europe'
}

# Hem 'first_orig_dist' hem de 'first_dest_dist' kolonlarına kıta bilgisi eklemek için map fonksiyonunu kullanıyoruz.
result['orig_continent'] = result['orig_dist'].map(continent_mapping)
result['dest_continent'] = result['dest_dist'].map(continent_mapping)

result["continental_type"] = result.apply(
    lambda row: "Within Europe" if row["orig_continent"] == "Europe" and row["dest_continent"] == "Europe"
    else "Within Asia" if row["orig_continent"] == "Asia" and row["dest_continent"] == "Asia"
    else "Intercontinental",
    axis=1
)

In [38]:
import pandas as pd

# 1. Intercontinental + geçerli purpose verilerini filtrele
filtered = result[
    (result['purpose'].notna()) &
    (result['purpose'] != 'Eve gidiş') &
    (result['continental_type'] == 'Intercontinental')
].copy()

# 2. purpose değerlerini gruplandır
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 3. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. purpose_group dağılımı (sayı ve yüzde)
purpose_counts = filtered['purpose_group'].value_counts().reset_index()
purpose_counts.columns = ['purpose_group', 'count']
total = purpose_counts['count'].sum()
purpose_counts['percentage'] = round((purpose_counts['count'] / total) * 100, 2)
purpose_counts

,purpose_group,count,percentage
0,Work,1702,65.97
1,School,430,16.67
2,Leisure,382,14.81
3,Health,47,1.82
4,Pick-up/Drop-off,19,0.74


In [39]:
import pandas as pd

# 1. Intercontinental + geçerli purpose verilerini filtrele
filtered = result[
    (result['purpose'].notna()) &
    (result['purpose'] != 'Eve gidiş') &
    (result['continental_type'] == 'Within Europe')
].copy()

# 2. purpose değerlerini gruplandır
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 3. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. purpose_group dağılımı (sayı ve yüzde)
purpose_counts = filtered['purpose_group'].value_counts().reset_index()
purpose_counts.columns = ['purpose_group', 'count']
total = purpose_counts['count'].sum()
purpose_counts['percentage'] = round((purpose_counts['count'] / total) * 100, 2)
purpose_counts

,purpose_group,count,percentage
0,Work,31115,39.86
1,Leisure,23011,29.48
2,School,15804,20.25
3,Pick-up/Drop-off,5246,6.72
4,Health,2877,3.69


In [40]:
import pandas as pd

# 1. Intercontinental + geçerli purpose verilerini filtrele
filtered = result[
    (result['purpose'].notna()) &
    (result['purpose'] != 'Eve gidiş') &
    (result['continental_type'] == 'Within Asia')
].copy()

# 2. purpose değerlerini gruplandır
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 3. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. purpose_group dağılımı (sayı ve yüzde)
purpose_counts = filtered['purpose_group'].value_counts().reset_index()
purpose_counts.columns = ['purpose_group', 'count']
total = purpose_counts['count'].sum()
purpose_counts['percentage'] = round((purpose_counts['count'] / total) * 100, 2)
purpose_counts

,purpose_group,count,percentage
0,Leisure,16122,38.67
1,Work,14246,34.17
2,School,7379,17.70
3,Pick-up/Drop-off,2142,5.14
4,Health,1802,4.32


<span style="font-size:22pt; color:red;"><b>Figure 103.Public Transport Usage Rates for İntercontinental Trips (İstanbul SUMP Stage II HTS)</b></span>

### 

In [41]:
istanbul_ilceleri = [
    "ADALAR", "ARNAVUTKÖY", "ATAŞEHİR", "AVCILAR", "BAĞCILAR", "BAHÇELİEVLER", "BAKIRKÖY",
    "BAŞAKŞEHİR", "BAYRAMPAŞA", "BEŞİKTAŞ", "BEYKOZ", "BEYLİKDÜZÜ", "BEYOĞLU", "BÜYÜKÇEKMECE",
    "ÇATALCA", "ÇEKMEKÖY", "ESENLER", "ESENYURT", "EYÜPSULTAN", "FATİH", "GAZİOSMANPAŞA",
    "GÜNGÖREN", "KADIKÖY", "KAĞITHANE", "KARTAL", "KÜÇÜKÇEKMECE", "MALTEPE", "PENDİK",
    "SANCAKTEPE", "SARIYER", "ŞİLE", "SİLİVRİ", "ŞİŞLİ", "SULTANBEYLİ", "SULTANGAZİ",
    "TUZLA", "ÜMRANİYE", "ÜSKÜDAR", "ZEYTİNBURNU"
]

# İstanbul ilçelerine ait satırları filtrele
result_istanbul_2 = result[result['orig_dist'].isin(istanbul_ilceleri)].copy()

# 1. Intercontinental yolculukları filtrele
intercontinental = result_istanbul_2[
    result_istanbul_2['continental_type'] == 'Intercontinental'
].copy()

# 2. mode_choice boş olmayanları al
intercontinental = intercontinental[intercontinental['mode_choice'].notna()]

# 3. İlçe ve taşıma moduna göre sayım tablosu
mode_counts = (
    intercontinental.groupby(['district', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik olabilecek kolonları garanti altına al
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in mode_counts.columns:
        mode_counts[col] = 0

# 5. Yüzdelik dağılımı hesapla (satır bazlı)
mode_percent = mode_counts.copy()
mode_cols = ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']
mode_percent[mode_cols] = mode_percent[mode_cols].div(mode_percent[mode_cols].sum(axis=1), axis=0) * 100
mode_percent[mode_cols] = mode_percent[mode_cols].round(2)

# 6. Sonucu birleştir (sayısal ve yüzdesel birlikte)
combined = pd.concat(
    [mode_counts.set_index('district'), mode_percent[mode_cols].add_suffix(" (%)").set_index(mode_counts['district'])],
    axis=1
).reset_index()

# 7. Genel toplam satırını hesapla
totals = mode_counts[mode_cols].sum()
percent_totals = (totals / totals.sum() * 100).round(2)

# 8. Toplam satırını oluştur
total_row = pd.DataFrame([{
    'district': 'Toplam',
    **totals.to_dict(),
    **{col + ' (%)': percent_totals[col] for col in mode_cols}
}])

# 9. Toplam satırı da dahil et
combined = pd.concat([combined, total_row], ignore_index=True)

# 10. Göster
combined.to_excel("103.xlsx")


<span style="font-size:22pt; color:red;"><b>Figure 107. Distribution of Transportation Modes According to Travel Purposes</b></span>

####
🧭 Şekil – Seyahat Amaçlarına Göre Ulaşım Modu Tercihleri

Bu tablo, her bir **seyahat amacı (purpose)** için hangi **ulaşım modlarının (mode_choice)** ne sıklıkta kullanıldığını göstermektedir. Tabloya ayrıca **en altta tüm modlar için toplam kullanım sayısı** da eklenmiştir.

#### Tablo Açıklaması:
- **purpose**: Yolculuğun amacı (örneğin: Alışveriş, İşe gidiş, Okula gidiş, vb.)
- **Service**: Servis aracıyla yapılan yolculuk sayısı
- **Private Vehicle**: Özel araçla yapılan yolculuk sayısı
- **Public Transport**: Toplu taşıma araçlarıyla yapılan yolculuk sayısı
- **Walking**: Yaya olarak yapılan yolculuk sayısı
- **Micromobility**: Scooter, bisiklet vb. mikro hareketlilik araçlarıyla yapılan yolculuk sayısı
- **Toplam**: Her modun toplammı (tüm ama

In [42]:
import pandas as pd

# 1. Geçerli verileri filtrele
filtered = result[
    (result['mode_choice'].notna()) &
    (result['purpose'].notna())
].copy()

# 2. Grup bazlı sayım: purpose x mode_choice
grouped = (
    filtered.groupby(['purpose', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 3. Eksik mod kolonlarını kontrol et ve ekle
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 4. Sadece istenen kolonları al
purpose_mode_counts = grouped[[
    'purpose', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

# 5. Toplam satırı ekle
total_row = purpose_mode_counts.drop(columns='purpose').sum(numeric_only=True)
total_row['purpose'] = 'Toplam'

purpose_mode_counts = pd.concat(
    [purpose_mode_counts, pd.DataFrame([total_row])],
    ignore_index=True
)

# 6. Yüzdesel dağılımı hesapla
percent_df = purpose_mode_counts.copy()
mode_columns = ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']
percent_df[mode_columns] = percent_df[mode_columns].div(
    percent_df[mode_columns].sum(axis=1), axis=0
) * 100

# 7. Amaçları da dahil et
percent_df['purpose'] = purpose_mode_counts['purpose']

# 8. Sonuçları göster
percent_df = percent_df[['purpose'] + mode_columns]
percent_df = percent_df.round(1)  # Yüzdelikleri 2 ondalıkla göster

percent_df.to_excel("testo.xlsx")



<span style="font-size:22pt; color:red;"><b>Figure 112. General Modal Split of Work Purposes Trips</b></span>


Bu tablo, yalnızca **"İşe gidiş"** amaçlı yapılan yolculuklar dikkate alınarak, tercih edilen ulaşım modlarının dağılımını göstermektedir.

#### Tablo Açıklaması:
- **mode_choice**: Kullanıcının tercih ettiği ulaşım modu (örneğin: Walking, Public Transport, Private Vehicle vb.).
- **count**: Belirli ulaşım modunun kaç kez kullanıldığını gösterir.
- **percentage**: İlgili modun "İşe gidiş" yolculukları içindeki yüzdesel oranı (%).

In [43]:
import pandas as pd

# 1. Filtreleme: sadece 'İşe gidiş' olanlar ve geçerli mode_choice'ler
filtered = result[
    (result['purpose'] == 'İşe gidiş') &
    (result['mode_choice'].notna())
].copy()

# 2. mode_choice bazında sayım
mode_counts = filtered['mode_choice'].value_counts().reindex(
    ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'],
    fill_value=0
).reset_index()

mode_counts.columns = ['mode_choice', 'count']

# 3. Yüzde ekle
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)
mode_counts

,mode_choice,count,percentage
0,Service,7525,15.99
1,Private Vehicle,15652,33.26
2,Public Transport,12324,26.19
3,Walking,11349,24.11
4,Micromobility,213,0.45


<span style="font-size:22pt; color:red;"><b>Figure 113-114-115.Distribution of transportation modes among employees commuting for work by workplace location in districts of Istanbul (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, sadece **"İşe gidiş"** amaçlı yapılan yolculuklar dikkate alınarak, her bir varış ilçesi (`dest_dist`) için tercih edilen ulaşım modlarının dağılımını göstermektedir.

#### Tablo Açıklaması:
- **dest_dist**: Yolculuğun varış noktası olan ilçe.
- **Service**: Servis aracı ile yapılan yolculukların sayısı.
- **Private Vehicle**: Özel araç ile yapılan yolculukların sayısı.
- **Public Transport**: Toplu taşıma ile yapılan yolculukların sayısı.
- **Walking**: Yaya olarak yapılan yolculukların sayısı.
- **Micromobility**: Bisiklet, scooter gibi mikro hareketlilik araçlarıyla yapılan yolculukların sayısı.



In [44]:
import pandas as pd

# 1. Filtre: 'İşe gidiş' ve mode_category dolu olanlar
filtered = result[
    (result['purpose'] == 'İşe gidiş') &
    (result['mode_choice'].notna())
].copy()

# 2. Grup bazlı sayım: destination x mode_category
grouped = (
    filtered.groupby(['dest_dist', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 3. Eksik mod kolonlarını ekle (varsa 0 ile)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 4. Kolonları sıralı hale getir
destination_mode_counts = grouped[[
    'dest_dist', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]
destination_mode_counts.to_excel("113.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 116-117-118.Distribution of transportation modes among employees commuting for work by home location in districts of Istanbul (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, yalnızca **"İşe gidiş"** amaçlı yolculukları baz alarak, her bir ilçede (`district`) hangi ulaşım modlarının ne sıklıkta tercih edildiğini göstermektedir.

#### Tablo Açıklaması:
- **district**: Yolculuğun başladığı ilçe.
- **Service**: Servis aracıyla yapılan yolculuk sayısı.
- **Private Vehicle**: Özel araçla yapılan yolculuk sayısı.
- **Public Transport**: Toplu taşıma ile yapılan yolculuk sayısı.
- **Walking**: Yaya olarak yapılan yolculuk sayısı.
- **Micromobility**: Bisiklet, scooter vb. mikro hareketlilik araçlarıyla yapılan yolculuk sayısı.

In [45]:
import pandas as pd

# 1. Filtre: 'İşe gidiş' ve geçerli mode_category olanlar
filtered = result[
    (result['purpose'] == 'İşe gidiş') &
    (result['mode_choice'].notna())
].copy()

# 2. Grup bazlı sayım: district x mode_category
grouped = (
    filtered.groupby(['district', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 3. Eksik mod kolonlarını sıfırla (eğer tabloda yoksa)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 4. Kolonları sıralı hale getir
district_mode_counts = grouped[[
    'district', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]
district_mode_counts.to_excel("116.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 119.Mode Choice Based on Income Levels (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, bireylerin ait olduğu **hane gelir grubuna (income_group)** göre hangi **ulaşım modlarını (mode_choice)** tercih ettiklerini göstermektedir. Hanehalkı gelir bilgileri, belirli aralıklarla tanımlanmış ve üç ana gruba ayrılmıştır:

#### Gelir Grupları:
- **Low**: 25.000 TL altı (düşük gelirli)
- **Medium**: 25.001 – 80.000 TL arası (orta gelirli)
- **High**: 80.001 TL ve üzeri (yüksek gelirli)
- **Not Specified**: Gelir bilgisi olmayanlar

#### Tablo Açıklaması:
- **income_group**: Yukarıdaki sınıflamaya göre hane gelir grubu
- **Service**: Servis aracı ile yapılan yolculuk sayısı
- **Private Vehicle**: Özel araçla yapılan yolculuk sayısı
- **Public Transport**: Toplu taşıma ile yapılan yolculuk sayısı
- **Walking**: Yaya olarak yapılan yolculuk sayısı
- **Micromobility**: Scooter, bisiklet gibi mikro hareketlilik araçlarıyla yapılan yolculuk sayısı



In [46]:
house = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Household.xlsx')

C:\Users\Paraboly\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [47]:
import pandas as pd

# 1. result ve house tablosunu hh_id üzerinden birleştir
merged = result.merge(house, on='hh_id', how='left')

# 2. Gelir grubu oluştur
def categorize_income(income):
    high = ['100.000TL’den fazla', '100.001 - 140.000 TL', '140.001 - 200.000 TL',
            '200.001TL’den fazla', '60.001 - 100.000 TL', '80.001 - 100.000 TL']
    medium = ['25.001 - 40.000 TL', '40.001 - 60.000 TL', '60.001 - 80.000 TL']
    low = ['5.000 TL’den az', '5.001 – 10.000 TL', '10.001 - 15.000 TL', '15.001 - 25.000 TL']
    
    if income in high:
        return 'High'
    elif income in medium:
        return 'Medium'
    elif income in low:
        return 'Low'
    else:
        return 'Not Specified'

merged['income_group'] = merged['hh_income'].apply(categorize_income)

# 3. mode_category boş olmayan ve gelir grubu belirlenmiş olanları filtrele
filtered = merged[merged['mode_choice'].notna()].copy()

# 4. Grup bazlı sayım: income_group x mode_category
grouped = (
    filtered.groupby(['income_group', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 5. Eksik mod kolonlarını sıfırla (varsa)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 6. Kolonları sıralı hale getir
income_mode_counts = grouped[[
    'income_group', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

# 7. İsteğe bağlı: Gelir grubu sıralaması
income_mode_counts['income_group'] = pd.Categorical(
    income_mode_counts['income_group'],
    categories=['Low', 'Medium', 'High', 'Not Specified'],
    ordered=True
)
income_mode_counts = income_mode_counts.sort_values('income_group').reset_index(drop=True)
income_mode_counts

mode_choice,income_group,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Low,4265,6453,12148,43703,230
1,Medium,10431,22681,22960,52962,403
2,High,853,4573,1419,4610,15
3,Not Specified,3568,9863,9289,23102,142


In [48]:
# 8. Yüzdesel dağılım ekle (satır bazında yüzde)
income_mode_percent = income_mode_counts.copy()

# Sadece sayı içeren kolonları normalize et ve 1 ondalık basamağa yuvarla
mode_cols = ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']
income_mode_percent[mode_cols] = (
    income_mode_percent[mode_cols].div(income_mode_percent[mode_cols].sum(axis=1), axis=0) * 100
).round(1)

# Sonuç
income_mode_percent


mode_choice,income_group,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Low,6.4,9.7,18.2,65.4,0.3
1,Medium,9.5,20.7,21.0,48.4,0.4
2,High,7.4,39.9,12.4,40.2,0.1
3,Not Specified,7.8,21.5,20.2,50.3,0.3


<span style="font-size:22pt; color:red;"><b>Figure 120.Vehicle Ownership and Mode Choices in High Income HTS</b></span>

Bu tablo, **yüksek gelir grubuna** dahil olan bireylerin araç sahipliği durumuna göre hangi ulaşım modlarını tercih ettiklerini göstermektedir.

#### Gelir Aralığı (High Group):
- 60.001 TL - 100.000 TL
- 80.001 TL - 100.000 TL
- 100.001 TL - 140.000 TL
- 140.001 TL - 200.000 TL
- 200.001 TL’den fazla

#### Tablo Açıklaması:
- **hh_has_vehicle**: Hane halkının aracı olup olmadığını belirtir (`Evet` / `Hayır`).
- **Service**: Servis aracıyla yapılan yolculuk sayısı.
- **Private Vehicle**: Özel araçla yapılan yolculuk sayısı.
- **Public Transport**: Toplu taşıma ile yapılan yolculuk sayısı.
- **Walking**: Yaya olarak yapılan yolculuk sayısı.
- **Micromobility**: Bisiklet, scooter gibi mikro hareketlilik araçlarıyla yapılan yolculuk sayısı.



In [49]:
import pandas as pd

# 1. result ve house tablolarını hh_id üzerinden birleştir
merged = result.merge(house, on='hh_id', how='left')

# 2. Düşük gelir grubunu tanımla
high = ['100.000TL’den fazla', '100.001 - 140.000 TL', '140.001 - 200.000 TL',
            '200.001TL’den fazla', '60.001 - 100.000 TL', '80.001 - 100.000 TL']


# 3. Filtrele: geçerli mode_category + düşük gelir grubundakiler
filtered = merged[
    (merged['mode_choice'].notna()) &
    (merged['hh_income'].isin(high))
].copy()

# 4. Grup bazlı sayım: hh_has_vehicle x mode_category
grouped = (
    filtered.groupby(['hh_has_vehicle', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 5. Eksik kolonları sıfırla (eğer veri setinde yoksa)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 6. Kolonları sıralı hale getir
vehicle_mode_counts = grouped[[
    'hh_has_vehicle', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

# 7. Araç sahipliği bilgisi sıralı hale getirmek istersen (isteğe bağlı)
# Örnek: 'Evet' ve 'Hayır' gibi kategoriler varsa:
vehicle_mode_counts['hh_has_vehicle'] = pd.Categorical(
    vehicle_mode_counts['hh_has_vehicle'],
    categories=['Hayır', 'Evet'],  # ihtiyaca göre değiştir
    ordered=True
)
vehicle_mode_counts = vehicle_mode_counts.sort_values('hh_has_vehicle').reset_index(drop=True)
vehicle_mode_counts

mode_choice,hh_has_vehicle,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Hayır,216,257,589,1344,1
1,Evet,637,4316,830,3266,14


In [50]:
# 8. Yüzdesel dağılım ekle (satır bazında yüzde, 1 ondalık basamakla)
vehicle_mode_percent = vehicle_mode_counts.copy()

mode_cols = ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']
vehicle_mode_percent[mode_cols] = (
    vehicle_mode_percent[mode_cols].div(vehicle_mode_percent[mode_cols].sum(axis=1), axis=0) * 100
).round(1)

# Sonuç
vehicle_mode_percent


mode_choice,hh_has_vehicle,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Hayır,9.0,10.7,24.5,55.8,0.0
1,Evet,7.0,47.6,9.2,36.0,0.2


<span style="font-size:22pt; color:red;"><b>Figure 121.Vehicle Ownership and Mode Choices in Low Income HTS</b></span>

Bu tablo, **düşük gelir grubuna** dahil olan bireylerin araç sahipliği durumuna göre hangi ulaşım modlarını tercih ettiklerini göstermektedir.

#### Gelir Aralığı (High Group):
- 60.001 TL - 100.000 TL
- 80.001 TL - 100.000 TL
- 100.001 TL - 140.000 TL
- 140.001 TL - 200.000 TL
- 200.001 TL’den fazla

#### Tablo Açıklaması:
- **hh_has_vehicle**: Hane halkının aracı olup olmadığını belirtir (`Evet` / `Hayır`).
- **Service**: Servis aracıyla yapılan yolculuk sayısı.
- **Private Vehicle**: Özel araçla yapılan yolculuk sayısı.
- **Public Transport**: Toplu taşıma ile yapılan yolculuk sayısı.
- **Walking**: Yaya olarak yapılan yolculuk sayısı.
- **Micromobility**: Bisiklet, scooter gibi mikro hareketlilik araçlarıyla yapılan yolculuk sayısı.



In [51]:
import pandas as pd

# 1. result ve house tablolarını hh_id üzerinden birleştir
merged = result.merge(house, on='hh_id', how='left')

# 2. Düşük gelir grubunu tanımla
low = ['5.000 TL’den az', '5.001 – 10.000 TL', '10.001 - 15.000 TL', '15.001 - 25.000 TL']

# 3. Filtrele: geçerli mode_category + düşük gelir grubundakiler
filtered = merged[
    (merged['mode_choice'].notna()) &
    (merged['hh_income'].isin(low))
].copy()

# 4. Grup bazlı sayım: hh_has_vehicle x mode_category
grouped = (
    filtered.groupby(['hh_has_vehicle', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 5. Eksik kolonları sıfırla (eğer veri setinde yoksa)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 6. Kolonları sıralı hale getir
vehicle_mode_counts = grouped[[
    'hh_has_vehicle', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

# 7. Araç sahipliği bilgisi sıralı hale getirmek istersen (isteğe bağlı)
# Örnek: 'Evet' ve 'Hayır' gibi kategoriler varsa:
vehicle_mode_counts['hh_has_vehicle'] = pd.Categorical(
    vehicle_mode_counts['hh_has_vehicle'],
    categories=['Hayır', 'Evet'],  # ihtiyaca göre değiştir
    ordered=True
)
vehicle_mode_counts = vehicle_mode_counts.sort_values('hh_has_vehicle').reset_index(drop=True)
vehicle_mode_counts

mode_choice,hh_has_vehicle,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Hayır,3417,2233,9979,35518,24
1,Evet,848,4220,2169,8185,206


In [52]:
# 8. Yüzdesel dağılım ekle (satır bazında yüzde, 1 ondalık basamakla)
vehicle_mode_percent = vehicle_mode_counts.copy()

mode_cols = ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']
vehicle_mode_percent[mode_cols] = (
    vehicle_mode_percent[mode_cols].div(vehicle_mode_percent[mode_cols].sum(axis=1), axis=0) * 100
).round(1)

# Sonuç
vehicle_mode_percent


mode_choice,hh_has_vehicle,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Hayır,6.7,4.4,19.5,69.4,0.0
1,Evet,5.4,27.0,13.9,52.4,1.3


<span style="font-size:22pt; color:red;"><b>Figure 122.HTS Mode Choices in Low Income District- BAĞCILAR</b></span>

Bu tablo, **BAĞCILAR** ilçesinde yapılan yolculuklar arasında tercih edilen ulaşım modlarının sayısal ve yüzdesel dağılımını göstermektedir.

#### Tablo Açıklaması:
- **mode_choice**: Tercih edilen ulaşım modu (ör. Walking, Public Transport, Private Vehicle, vb.)
- **count**: İlgili modun kaç defa tercih edildiği
- **percentage**: Toplam yolculuklar içindeki oranı (%)

In [53]:
import pandas as pd

# 1. Filtrele: BEŞİKTAŞ ilçesi ve geçerli mode_choice
filtered = result[
    (result['district'] == 'BAĞCILAR') &
    (result['mode_choice'].notna())
].copy()

# 2. mode_choice bazında sayım
mode_counts = filtered['mode_choice'].value_counts().reindex(
    ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'],
    fill_value=0
).reset_index()

# 3. Kolon adlarını ayarla
mode_counts.columns = ['mode_choice', 'count']

# 4. Yüzdesel dağılım hesapla
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)
mode_counts

,mode_choice,count,percentage
0,Service,1317,10.12
1,Private Vehicle,1629,12.52
2,Public Transport,2386,18.34
3,Walking,7668,58.93
4,Micromobility,12,0.09


<span style="font-size:22pt; color:red;"><b>Figure 123.HTS Mode Choices in Low Income District- BEŞİKTAŞ</b></span>

Bu tablo, **BEŞİKTAŞ** ilçesinde yapılan yolculuklar arasında tercih edilen ulaşım modlarının sayısal ve yüzdesel dağılımını göstermektedir.

#### Tablo Açıklaması:
- **mode_choice**: Tercih edilen ulaşım modu (ör. Walking, Public Transport, Private Vehicle, vb.)
- **count**: İlgili modun kaç defa tercih edildiği
- **percentage**: Toplam yolculuklar içindeki oranı (%)

In [54]:
import pandas as pd

# 1. Filtrele: BEŞİKTAŞ ilçesi ve geçerli mode_choice
filtered = result[
    (result['district'] == 'BEŞİKTAŞ') &
    (result['mode_choice'].notna())
].copy()

# 2. mode_choice bazında sayım
mode_counts = filtered['mode_choice'].value_counts().reindex(
    ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'],
    fill_value=0
).reset_index()

# 3. Kolon adlarını ayarla
mode_counts.columns = ['mode_choice', 'count']

# 4. Yüzdesel dağılım hesapla
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)
mode_counts

,mode_choice,count,percentage
0,Service,91,4.05
1,Private Vehicle,659,29.30
2,Public Transport,447,19.88
3,Walking,999,44.42
4,Micromobility,53,2.36


<span style="font-size:22pt; color:red;"><b>Figure 124.Modal split of university-location based educational trips based on districts of İstanbul (İstanbul SUMP Stage II HTS) </b></span>

Bu tablo, üniversite amacıyla yapılan yolculuklarda, varış bölgesi (dest_dist) bazında tercih edilen ulaşım modlarının sayısal dağılımını göstermektedir.


#### Tablo Açıklaması:
**dest_dist**: Yolculuğun sona erdiği (varış) bölgeyi belirtir.

**Service**: Servis aracıyla yapılan yolculuk sayısı.

**Private Vehicle**: Özel araçla yapılan yolculuk sayısı.

**Public Transport**: Toplu taşıma ile yapılan yolculuk sayısı.

**Walking**: Yürüyerek yapılan yolculuk sayısı.

**Micromobility**: Bisiklet, scooter vb. mikro mobilite araçlarıyla yapılan yolculuk sayısı.

In [55]:
import pandas as pd

# 1. Okula gidiş türlerini tanımla
school_purposes = [
    'Okula gidiş(üniversite ve üzeri)'
]

# 2. Filtreleme
filtered = result[
    (result['mode_choice'].notna()) &
    (result['purpose'].isin(school_purposes))
].copy()

# 3. Grup bazlı sayım
grouped = (
    filtered.groupby(['dest_dist', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik kolonları sıfırla
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Kolonları sırala
destination_mode_counts = grouped[[
    'dest_dist', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

# 6. Sonucu göster
destination_mode_counts.to_excel("124.xlsx")


<span style="font-size:22pt; color:red;"><b>Figure 125-126-127.Modal split of school location based educational trips among districts of İstanbul (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, okula gidiş (ilk-ortaöğretim, lise, üniversite ve üzeri) amacıyla yapılan yolculuklarda, varış bölgesi (dest_dist) bazında tercih edilen ulaşım modlarının sayısal dağılımını göstermektedir.


#### Tablo Açıklaması:
**dest_dist**: Yolculuğun sona erdiği (varış) bölgeyi belirtir.

**Service**: Servis aracıyla yapılan yolculuk sayısı.

**Private Vehicle**: Özel araçla yapılan yolculuk sayısı.

**Public Transport**: Toplu taşıma ile yapılan yolculuk sayısı.

**Walking**: Yürüyerek yapılan yolculuk sayısı.

**Micromobility**: Bisiklet, scooter vb. mikro mobilite araçlarıyla yapılan yolculuk sayısı.

In [56]:
import pandas as pd

# 1. Okula gidiş türlerini tanımla
school_purposes = [
    'Okula gidiş(ilk ve ortaöğretim)',
    'Okula gidiş(lise)'
]

# 2. Filtreleme
filtered = result[
    (result['mode_choice'].notna()) &
    (result['purpose'].isin(school_purposes))
].copy()

# 3. Grup bazlı sayım
grouped = (
    filtered.groupby(['dest_dist', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik kolonları sıfırla
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Kolonları sırala
destination_mode_counts = grouped[[
    'dest_dist', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

# 6. Sonucu göster
destination_mode_counts.to_excel("125.xlsx")


<span style="font-size:22pt; color:red;"><b>Figure 128.Modal split of education purpose travel with home location based on districts of İstanbul (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, **okula gidiş** amacıyla yapılan yolculuklarda (ilk ve ortaöğretim, lise, üniversite ve üzeri), yolculuğun başladığı **ilçeye (district)** göre tercih edilen ulaşım modlarının sayısal dağılımını göstermektedir.

#### 📋 Tablo Açıklaması:
- **district**: Yolculuğun başladığı ilçe
- **Service**: Servis aracı ile yapılan yolculukların sayısı
- **Private Vehicle**: Özel araç ile yapılan yolculukların sayısı
- **Public Transport**: Toplu taşıma ile yapılan yolculukların sayısı
- **Walking**: Yürüyerek yapılan yolculukların sayısı
- **Micromobility**: Scooter, bisiklet gibi mikro mobilite araçları ile yapılan yolculukların sayısı

In [57]:
school_purposes = [
    'Okula gidiş(ilk ve ortaöğretim)',
    'Okula gidiş(lise)',
    'Okula gidiş(üniversite ve üzeri)'
]

# 2. Geçerli verileri filtrele
filtered = result[
    (result['mode_choice'].notna()) &
    (result['purpose'].isin(school_purposes))
].copy()

# 3. Grup bazlı sayım: district x mode_choice
grouped = (
    filtered.groupby(['district', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik kolonları sıfırla (eksik modlar varsa)
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Kolonları sıralı hale getir
district_mode_counts = grouped[[
    'district', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]
district_mode_counts.to_excel("128.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 131. Education-based Start Travel Time (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, **okula gidiş** amacıyla yapılan yolculuklarda (ilk ve ortaöğretim, lise, üniversite ve üzeri), yolculuğun başladığı **saate (start time)** göre tercih edilen ulaşım modlarının sayısal dağılımını göstermektedir.

#### 📋 Tablo Açıklaması:
- **start time**: Yolculuğun başladığı saat (örneğin: 08:00)
- **Primary School**: İlk ve ortaöğretim için yapılan yolculuk sayısı
- **High School**: Lise için yapılan yolculuk sayısı
- **University**: Üniversite ve üzeri için yapılan yolculuk sayısı
n sayısı

In [58]:
import pandas as pd

# 1. Okula gidiş amaçlarını tanımla
school_purposes = [
    'Okula gidiş(ilk ve ortaöğretim)',
    'Okula gidiş(lise)',
    'Okula gidiş(üniversite ve üzeri)'
]

# 2. Geçerli mode_choice ve okula gidiş verilerini filtrele
filtered = result[
    (result['mode_choice'].notna()) &
    (result['purpose'].isin(school_purposes))
].copy()

# 3. Saatlik grup kolonu oluştur (ör. '08:00')
filtered['hour_group'] = pd.to_datetime(filtered['start_time'], errors='coerce').dt.strftime('%H:00')

# 4. Her okul türü için sayım
hourly_counts = (
    filtered.groupby(['hour_group', 'purpose'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 5. Eksik kolonlar varsa sıfırla
for col in [
    'Okula gidiş(ilk ve ortaöğretim)',
    'Okula gidiş(lise)',
    'Okula gidiş(üniversite ve üzeri)'
]:
    if col not in hourly_counts.columns:
        hourly_counts[col] = 0

# 6. Kolonları yeniden adlandır
hourly_counts = hourly_counts.rename(columns={
    'Okula gidiş(ilk ve ortaöğretim)': 'Primary School',
    'Okula gidiş(lise)': 'High School',
    'Okula gidiş(üniversite ve üzeri)': 'University'
})

# 7. Saat sırasına göre sırala
hourly_counts = hourly_counts.sort_values('hour_group').reset_index(drop=True)

# Sonuç:
hourly_counts


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_5020\584057547.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered['hour_group'] = pd.to_datetime(filtered['start_time'], errors='coerce').dt.strftime('%H:00')


purpose,hour_group,Primary School,High School,University
0,00:00,3,0,0
1,01:00,3,0,0
2,04:00,0,1,1
3,05:00,1,5,7
4,06:00,238,296,122
5,07:00,5139,2690,504
6,08:00,6178,2504,831
7,09:00,382,168,652
8,10:00,43,18,512
9,11:00,103,49,212


In [59]:
# 8. Kolon (okul türü) bazlı yüzdesel dağılım (sütun toplamına göre)
hourly_percent = hourly_counts.copy()

# Yüzdelik dönüşüm: Her kolonu kendi toplamına böl ve % olarak hesapla
school_cols = ['Primary School', 'High School', 'University']
hourly_percent[school_cols] = (
    hourly_percent[school_cols].div(hourly_percent[school_cols].sum(axis=0), axis=1) * 100
).round(1)

# Sonuç
hourly_percent


purpose,hour_group,Primary School,High School,University
0,00:00,0.0,0.0,0.0
1,01:00,0.0,0.0,0.0
2,04:00,0.0,0.0,0.0
3,05:00,0.0,0.1,0.2
4,06:00,1.7,4.9,3.8
5,07:00,35.8,44.4,15.8
6,08:00,43.0,41.3,26.1
7,09:00,2.7,2.8,20.5
8,10:00,0.3,0.3,16.1
9,11:00,0.7,0.8,6.7


<span style="font-size:22pt; color:red;"><b>Figure 132.Mode choice based on Person Types</b></span>

Bu tablo, **çalışma durumu (Employment Group)** farklılıklarına göre yapılan yolculuklarda tercih edilen ulaşım modlarının sayısal dağılımını göstermektedir.

#### 📋 Tablo Açıklaması:
- **Employment_Group**: Kişinin istihdam durumu (örneğin: Çalışıyor, İşsiz, Öğrenci vb.)
- **Service**: Servis aracı ile yapılan yolculukların sayısı
- **Private Vehicle**: Özel araç ile yapılan yolculukların sayısı
- **Public Transport**: Toplu taşıma ile yapılan yolculukların sayısı
- **Walking**: Yürüyerek yapılan yolculukların sayısı
- **Micromobility**: Scooter, bisiklet gibi mikro mobilite araçları ile yapılan yolculukların sayısısayısı

In [60]:
# 1. mode_choice ve employment_group geçerli olanları filtrele
filtered = result[
    result['mode_choice'].notna() &
    result['Employment_Group'].notna()
].copy()

# 2. Grup bazlı sayım
grouped = (
    filtered.groupby(['Employment_Group', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 3. Eksik kolonları sıfırla
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 4. Kolonları sıralı hale getir
employment_mode_counts = grouped[[
    'Employment_Group', 'Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'
]]

employment_mode_counts


mode_choice,Employment_Group,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,65+ Retired,26,1830,2063,13019,60
1,Adult Student,456,1277,6316,2616,32
2,Adult Unemployed,155,5625,7740,44895,100
3,Child,46,79,151,908,7
4,Child Student,3883,1829,4859,31621,124
5,Full Time,14320,31741,23692,26027,455
6,Other,25,464,332,4333,1
7,Part Time,206,725,663,958,11


In [61]:
# 5. Yüzdesel dağılım: satır bazlı (Employment_Group içinde mode_choice yüzdeleri)
employment_mode_percent = employment_mode_counts.copy()

mode_cols = ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']
employment_mode_percent[mode_cols] = (
    employment_mode_percent[mode_cols].div(employment_mode_percent[mode_cols].sum(axis=1), axis=0) * 100
).round(1)

# Sonuç
employment_mode_percent.to_excel("134.xlsx")


<span style="font-size:22pt; color:red;"><b>Figure 133-137. Trip Purpose Distribution of Age (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, belirli yaş aralığındaki bireylerin **‘Eve gidiş’ dışındaki** seyahatlerinde tercih ettikleri amaçlara göre yapılan yolculuk sayılarını göstermektedir.

#### 📋 Tablo Açıklaması:
- **purpose_group**: Yolculuğun amacı (örneğin: işe gidiş, sağlık kurumuna gidiş vb.)
- **trip_count**: İlgili amaçla yapılan yolculuk sayısı

In [62]:
profile = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\Anket Verisi\\HTS\\Profile.xlsx')

In [63]:
# hh_id ve person_order'ı birleştirerek yeni bir kolon oluştur
profile['person_id'] = profile['hh_id'].astype(str) + '-' + profile['person_order'].astype(str)
profile.head(2)

,p_id,hh_id,participation_status,person_order,name,age,gender,District,edu,drive_lic,house_rep_prox,work_status,work_type,sector,job,work_time,office_info,work_days,pt_card,pt_card_type,disability,had_trip,no_trip_reason,person_id
0,71,6kaqzPZwvLgUnuUalhTx,Evet,K1,Şükran Çiftçi,64.0,Kadın,BEYKOZ,İlköğretim mezunu,Yok,Kendisi,Çalışmıyor,NaN,NaN,NaN,NaN,NaN,0.0,Yok,NaN,Hayır,Hayır,Gerek Olmaması,6kaqzPZwvLgUnuUalhTx-K1
1,72,6kaqzPZwvLgUnuUalhTx,Evet,K2,Mahmut,67.0,Erkek,BEYKOZ,İlköğretim mezunu,Yok,Eşi,Çalışmıyor,NaN,NaN,NaN,NaN,NaN,0.0,Var,Ücretsiz Kart,Hayır,Evet,NaN,6kaqzPZwvLgUnuUalhTx-K2


In [64]:
import pandas as pd

# 0. p_id kolonlarını stringe çevir

# 1. Trip ve profile tablolarını p_id üzerinden birleştir
merged = result.merge(profile, on='person_id', how='left')

# 2. Filtreleme: purpose 'Eve gidiş' olmayanlar ve yaş >= 65
filtered = merged[
    (merged['purpose'].notna()) &
    (merged['purpose'] != 'Eve gidiş') &
    (merged['age'] >= 65)
].copy()

# 3. Amaçları grupla
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 4. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. Grup bazında sayım
grouped = (
    filtered.groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
    .sort_values('trip_count', ascending=False)
    .reset_index(drop=True)
)

grouped


,purpose_group,trip_count
0,Leisure,8479
1,Health,1614
2,Work,303
3,Pick-up/Drop-off,80
4,School,9


In [65]:
import pandas as pd

# 1. Trip ve profile tablolarını p_id üzerinden birleştir
merged = result.merge(profile, on='person_id', how='left')

# 2. Filtreleme: purpose 'Eve gidiş' olmayanlar ve yaş 0-14 arası olanlar
filtered = merged[
    (merged['purpose'].notna()) &
    (merged['purpose'] != 'Eve gidiş') &
    (merged['age'] >= 0) &
    (merged['age'] <= 14)
].copy()

# 3. Amaçları grupla
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 4. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. Grup bazında sayım
grouped = (
    filtered.groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
    .sort_values('trip_count', ascending=False)
    .reset_index(drop=True)
)

grouped


,purpose_group,trip_count
0,School,13658
1,Pick-up/Drop-off,2976
2,Leisure,2928
3,Work,279
4,Health,244


In [66]:
import pandas as pd

# 1. Trip ve profile tablolarını p_id üzerinden birleştir
merged = result.merge(profile, on='person_id', how='left')

# 2. Filtreleme: purpose 'Eve gidiş' olmayanlar ve yaş 15-29 arası olanlar
filtered = merged[
    (merged['purpose'].notna()) &
    (merged['purpose'] != 'Eve gidiş') &
    (merged['age'] >= 15) &
    (merged['age'] <= 29)
].copy()

# 3. Amaçları grupla
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 4. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. Grup bazında sayım
grouped = (
    filtered.groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
    .sort_values('trip_count', ascending=False)
    .reset_index(drop=True)
)

grouped


,purpose_group,trip_count
0,Work,12524
1,School,8781
2,Leisure,7411
3,Pick-up/Drop-off,631
4,Health,507


In [67]:
import pandas as pd

# 1. Trip ve profile tablolarını p_id üzerinden birleştir
merged = result.merge(profile, on='person_id', how='left')

# 2. Filtreleme: purpose 'Eve gidiş' olmayanlar ve yaş 30-44 arası olanlar
filtered = merged[
    (merged['purpose'].notna()) &
    (merged['purpose'] != 'Eve gidiş') &
    (merged['age'] >= 30) &
    (merged['age'] <= 44)
].copy()

# 3. Amaçları grupla
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 4. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. Grup bazında sayım
grouped = (
    filtered.groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
    .sort_values('trip_count', ascending=False)
    .reset_index(drop=True)
)
grouped


,purpose_group,trip_count
0,Work,22819
1,Leisure,8146
2,Pick-up/Drop-off,3111
3,School,976
4,Health,678


In [68]:
import pandas as pd

# 1. Trip ve profile tablolarını p_id üzerinden birleştir
merged = result.merge(profile, on='person_id', how='left')

# 2. Filtreleme: purpose 'Eve gidiş' olmayanlar ve yaş 45-64 arası olanlar
filtered = merged[
    (merged['purpose'].notna()) &
    (merged['purpose'] != 'Eve gidiş') &
    (merged['age'] >= 45) &
    (merged['age'] <= 64)
].copy()

# 3. Amaçları grupla
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 4. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. Grup bazında sayım
grouped = (
    filtered.groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
    .sort_values('trip_count', ascending=False)
    .reset_index(drop=True)
)

grouped


,purpose_group,trip_count
0,Leisure,12551
1,Work,11138
2,Health,1683
3,Pick-up/Drop-off,609
4,School,189


<span style="font-size:22pt; color:red;"><b>Figure 141-142-143-145-146.  Female's-Male's Travel Mode and Purpose Distribution (İstanbul SUMP Stage II HTS</b></span>

Bu tablo, **kadın** ve **erkek** yaptığı yolculuklarda tercih ettikleri ulaşım modlarının sayısal ve yüzdesel dağılımını göstermektedir.

#### 📋 Tablo Açıklaması:
- **mode_choice**: Tercih edilen ulaşım modu (ör. Walking, Public Transport, Private Vehicle, vb.)
- **count**: İlgili modun kaç defa tercih edildiği
- **percentage**: Toplam yolculuklar içindeki oranı (%)

In [69]:

# 3. Filtreleme: sadece kadınlar ve geçerli mode_choice
filtered = result[
    (result['mode_choice'].notna()) &
    (result['gender'] == 'Kadın')
].copy()

# 4. Ulaşım modu bazında sayım
mode_counts = filtered['mode_choice'].value_counts().reindex(
    ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'],
    fill_value=0
).reset_index()

# 5. Kolon adlarını düzenle
mode_counts.columns = ['mode_choice', 'count']

# 6. Yüzdesel dağılımı hesapla
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)

# 7. Sonuçları göster
mode_counts


,mode_choice,count,percentage
0,Service,5779,5.72
1,Private Vehicle,11045,10.92
2,Public Transport,18999,18.79
3,Walking,65124,64.42
4,Micromobility,153,0.15


In [70]:
import pandas as pd



# 2. Filtreleme: yaş ≥ 65, cinsiyet Kadın, purpose geçerli ve Eve gidiş olmayanlar
filtered = result[
    (result['gender'] == 'Kadın') &
    (result['purpose'].notna()) &
    (result['purpose'] != 'Eve gidiş')
].copy()

# 3. Amaçları gruplamak için fonksiyon
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 4. Yeni kolon oluştur: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. İlçe ve purpose_group bazında sayım
grouped_counts = (
    filtered.groupby(['district', 'purpose_group'])
    .size()
    .unstack(fill_value=0)
)

# 6. Yüzdesel dağılımı hesapla
grouped_percentages = (grouped_counts.T / grouped_counts.sum(axis=1)).T * 100
grouped_percentages = grouped_percentages.round(2).add_suffix(' (%)')

# 7. Sayım ve yüzde tablolarını birleştir
final_table = pd.concat([grouped_counts, grouped_percentages], axis=1)

# 8. Toplam sayım satırı
totals_row = pd.DataFrame([grouped_counts.sum()], index=['TOPLAM'])

# 9. Yüzdesel toplam (her sütun toplamı %100 olacak şekilde)
totals_percentage = pd.DataFrame([grouped_percentages.mean()], index=['TOPLAM'])

# 10. Sayım ve yüzde kolonlarını birleştir
total_row_combined = pd.concat([totals_row, totals_percentage], axis=1)

# 11. Son tabloya ekle
final_table_with_total = pd.concat([final_table, total_row_combined])

# 12. Sonucu göster
final_table_with_total.tail(3)  # Son 3 satırla kontrol edebilirsin (TOPLAM satırı dahil)

final_table_with_total.to_excel("Kadın.xlsx")

In [71]:
import pandas as pd

filtered = result[
    (result['mode_choice'].notna()) &
    (result['gender'] == 'Erkek')
].copy()

# 4. Ulaşım modu bazında sayım
mode_counts = filtered['mode_choice'].value_counts().reindex(
    ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'],
    fill_value=0
).reset_index()

# 5. Kolon adlarını düzenle
mode_counts.columns = ['mode_choice', 'count']

# 6. Yüzdesel dağılımı hesapla
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)

# 7. Sonuçları göster
mode_counts

,mode_choice,count,percentage
0,Service,13226,10.06
1,Private Vehicle,32410,24.64
2,Public Transport,26649,20.26
3,Walking,58590,44.55
4,Micromobility,637,0.48


In [72]:

# 2. Filtreleme: yaş ≥ 65, cinsiyet Erkek, purpose geçerli ve Eve gidiş olmayanlar
filtered = result[
    (result['gender'] == 'Erkek') &
    (result['purpose'].notna()) &
    (result['purpose'] != 'Eve gidiş')
].copy()

# 3. Amaçları gruplamak için fonksiyon
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 4. Yeni kolon oluştur: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. İlçe ve purpose_group bazında sayım
grouped_counts = (
    filtered.groupby(['district', 'purpose_group'])
    .size()
    .unstack(fill_value=0)
)

# 6. Yüzdesel dağılımı hesapla
grouped_percentages = (grouped_counts.T / grouped_counts.sum(axis=1)).T * 100
grouped_percentages = grouped_percentages.round(2).add_suffix(' (%)')

# 7. Sayım ve yüzde tablolarını birleştir
final_table = pd.concat([grouped_counts, grouped_percentages], axis=1)

# 8. Toplam sayım satırı
totals_row = pd.DataFrame([grouped_counts.sum()], index=['TOPLAM'])

# 9. Yüzdesel toplam (her sütun toplamı %100 olacak şekilde)
totals_percentage = pd.DataFrame([grouped_percentages.mean()], index=['TOPLAM'])

# 10. Sayım ve yüzde kolonlarını birleştir
total_row_combined = pd.concat([totals_row, totals_percentage], axis=1)

# 11. Son tabloya ekle
final_table_with_total = pd.concat([final_table, total_row_combined])

# 12. Sonucu göster
final_table_with_total.tail(3)  # Son 3 satırla kontrol edebilirsin (TOPLAM satırı dahil)
final_table_with_total.to_excel("Erkek.xlsx")

<span style="font-size:22pt; color:red;"><b>Figure 147. Leisure purposes of different person types (İstanbul SUMP Stage II HTS)</b></span>

### 

Bu tablo,eve gidiş dışındaki yolculuklarda, çalışma durumlarına (Employment Group) göre tercih ettikleri eğlence ve sosyal amaçlı yolculuk türlerinin sayısal dağılımını göstermektedir.

📋 **Tablo Açıklaması**:

**Employment_Group**: Bireyin çalışıp çalışmadığına dair grubu (örneğin: Çalışıyor, İşsiz, Öğrenci vb.)

**Spor, Eğlence vb. gidiş**: Spor, eğlence, etkinlik gibi nedenlerle yapılan yolculuk sayısı

**Yeme/içme Gidiş**: Restoran, kafe gibi yerlere yapılan yolculuk sayısı

**Alışveriş**: Market, mağaza gibi yerlere yapılan alışveriş amaçlı yolculuk sayısı

**Akraba/Arkadaş Ziyareti**: Sosyal ziyaret amaçlı yapılan yolculuk sayısı

**İş takibi/Ödemeler**: Resmi işler, fatura ödeme gibi nedenlerle yapılan yolculuk sayısı

**Diğer**: Yukarıdaki kategorilere uymayan diğer sosyal amaçlı yolculuklar

In [73]:
import pandas as pd

# 1. Trip ve profile tablolarını p_id (ya da sizin dediğiniz gibi 'person_id') üzerinden birleştir
merged = result.merge(profile, on='person_id', how='left')


# 2. Eve gidiş olmayan ve cinsiyeti Erkek olanları filtrele
filtered = merged[
    (merged['purpose'] != 'Eve gidiş')
    
].copy()

# 3. Amaçları grupla: employment_group x first_purpose (ya da purpose)
grouped = (
    filtered.groupby(['Employment_Group', 'purpose'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik kolonlar varsa sıfırla
columns_to_include = [
    'Spor, Eğlence vb. gidiş',
    'Yeme/içme Gidiş',
    'Alışveriş',
    'Akraba/Arkadaş  Ziyareti',
    'İş takibi/ Ödemeler',
    'Diğer'
]

for col in columns_to_include:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Kolonları sıralı hale getir
leisure_purpose_counts = grouped[[
    'Employment_Group',
    'Spor, Eğlence vb. gidiş',
    'Yeme/içme Gidiş',
    'Alışveriş',
    'Akraba/Arkadaş  Ziyareti',
    'İş takibi/ Ödemeler',
    'Diğer'
]]

# 6. Sonuçları göster
leisure_purpose_counts


purpose,Employment_Group,"Spor, Eğlence vb. gidiş",Yeme/içme Gidiş,Alışveriş,Akraba/Arkadaş Ziyareti,İş takibi/ Ödemeler,Diğer
0,65+ Retired,1142,344,3749,1232,137,799
1,Adult Student,211,100,309,82,12,111
2,Adult Unemployed,2969,1073,13384,3600,489,2035
3,Child,80,20,159,53,4,80
4,Child Student,174,36,234,105,4,421
5,Full Time,601,442,1772,369,157,605
6,Other,468,66,1117,291,10,123
7,Part Time,60,21,141,44,25,55


In [74]:
# 7. Satır bazlı yüzdesel dağılım (Employment_Group içindeki amaçların yüzdeleri)
leisure_purpose_percent = leisure_purpose_counts.copy()

# Sadece sayı içeren kolonlar
purpose_cols = [
    'Spor, Eğlence vb. gidiş',
    'Yeme/içme Gidiş',
    'Alışveriş',
    'Akraba/Arkadaş  Ziyareti',
    'İş takibi/ Ödemeler',
    'Diğer'
]

# Satır toplamına göre yüzde hesapla
leisure_purpose_percent[purpose_cols] = (
    leisure_purpose_percent[purpose_cols].div(
        leisure_purpose_percent[purpose_cols].sum(axis=1), axis=0
    ) * 100
).round(1)

# Sonuç
leisure_purpose_percent


purpose,Employment_Group,"Spor, Eğlence vb. gidiş",Yeme/içme Gidiş,Alışveriş,Akraba/Arkadaş Ziyareti,İş takibi/ Ödemeler,Diğer
0,65+ Retired,15.4,4.6,50.6,16.6,1.9,10.8
1,Adult Student,25.6,12.1,37.5,9.9,1.5,13.5
2,Adult Unemployed,12.6,4.6,56.8,15.3,2.1,8.6
3,Child,20.2,5.1,40.2,13.4,1.0,20.2
4,Child Student,17.9,3.7,24.0,10.8,0.4,43.2
5,Full Time,15.2,11.2,44.9,9.4,4.0,15.3
6,Other,22.6,3.2,53.8,14.0,0.5,5.9
7,Part Time,17.3,6.1,40.8,12.7,7.2,15.9


<span style="font-size:22pt; color:red;"><b>Figure 148.  Transport modes of different types of people for leisure trip purposes (İstanbul SUMP Stage II HTS)</b></span>

### 
Bu tablo, eğlence ve sosyal amaçlarla (örneğin spor, yeme-içme, alışveriş, arkadaş/akraba ziyareti, iş takibi, diğer) yapılan yolculuklarda, bireylerin çalışma durumuna (Employment Group) göre tercih ettikleri ulaşım modlarının sayısal dağılımını göstermektedir.

📋 **Tablo Açıklaması**:
**Employment_Group**: Bireyin çalışıp çalışmadığına dair grubu (örneğin: Çalışıyor, İşsiz, Öğrenci vb.)

**Service**: Servis aracıyla yapılan yolculuk sayısı

**Private Vehicle**: Özel araçla yapılan yolculuk sayısı

**Public Transport**: Toplu taşıma ile yapılan yolculuk sayısı

**Walking**: Yürüyerek yapılan yolculuk sayısı

**Micromobility**: Scooter, bisiklet gibi mikro mobilite araçlarıyla yapılan yolculuk sayısı(%)  

In [75]:
import pandas as pd

# 1. Trip ve profile verilerini birleştir (p_id üzerinden)
merged = result.merge(profile, on='person_id', how='left')

# 2. Eğlence ve sosyal amaçları tanımla
leisure_purposes = [
    'Spor, Eğlence vb. gidiş',
    'Yeme/içme Gidiş',
    'Alışveriş',
    'Akraba/Arkadaş  Ziyareti',
    'İş takibi/ Ödemeler',
    'Diğer'
]

# 3. Filtreleme: geçerli mode_choice ve amaç listedekiler
filtered = merged[
    (merged['mode_choice'].notna()) &
    (merged['purpose'].isin(leisure_purposes))
].copy()

# 4. Grup bazlı sayım: employment_group x mode_choice
grouped = (
    filtered.groupby(['Employment_Group', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 5. Eksik mod kolonlarını sıfırla
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 6. Kolonları sıralı hale getir
employment_leisure_modes = grouped[[
    'Employment_Group',
    'Service',
    'Private Vehicle',
    'Public Transport',
    'Walking',
    'Micromobility'
]]

# 7. Sonuçları göster
employment_leisure_modes


mode_choice,Employment_Group,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,65+ Retired,2,520,599,6252,30
1,Adult Student,2,73,216,527,7
2,Adult Unemployed,38,1959,2862,18640,51
3,Child,2,19,46,326,3
4,Child Student,46,88,140,696,4
5,Full Time,109,978,513,2321,25
6,Other,1,142,126,1806,0
7,Part Time,2,71,70,201,2


In [76]:
# 8. Transpose (satır-sütun takası) ve index'i mode_choice yap
transposed = employment_leisure_modes.set_index('Employment_Group').T.reset_index()
transposed = transposed.rename(columns={'index': 'mode_choice'})

# 9. Kolon bazlı yüzdesel dağılım (yani her Employment_Group içinde yüzdeler toplamı ≈ 100)
percent_df = transposed.copy()
employment_groups = percent_df.columns[1:]  # ilk kolon mode_choice olduğu için
percent_df[employment_groups] = (
    percent_df[employment_groups].div(percent_df[employment_groups].sum(axis=0), axis=1) * 100
).round(1)

# 10. Sonuç
percent_df


Employment_Group,mode_choice,65+ Retired,Adult Student,Adult Unemployed,Child,Child Student,Full Time,Other,Part Time
0,Service,0.0,0.2,0.2,0.5,4.7,2.8,0.0,0.6
1,Private Vehicle,7.0,8.8,8.3,4.8,9.0,24.8,6.8,20.5
2,Public Transport,8.1,26.2,12.2,11.6,14.4,13.0,6.1,20.2
3,Walking,84.5,63.9,79.2,82.3,71.5,58.8,87.0,58.1
4,Micromobility,0.4,0.8,0.2,0.8,0.4,0.6,0.0,0.6


<span style="font-size:22pt; color:red;"><b>Figure 150.Mode choice of Disabled People (İstanbul SUMP Stage II HTS)</b></span>
#### 
Bu tablo, **engelli bireylerin** yaptığı yolculuklarda tercih ettikleri ulaşım modlarının sayısal ve yüzdesel dağılımını göstermektedir.

 # 📋 Tablo Açıklaması:
- **mode_choice**: Tercih edilen ulaşım modu (ör. **Walking**, **Public Transport**, **Private Vehicle** vb.)
- **count**: İlgili modun kaç defa tercih edildiği
- **percentage**: Toplam yolculuklar içindeki oranı (%)

In [77]:
import pandas as pd
# 1. Trip ve profile verilerini birleştir (person_id üzerinden)
merged = result.merge(profile, on='person_id', how='left')

# 2. Filtreleme: mode_choice geçerli ve birey "disability" kolonu "Evet"
filtered = merged[
    (merged['mode_choice'].notna()) & 
    (merged['disability'] == 'Evet')
].copy()

# 3. Ulaşım modu bazında sayım
mode_counts = filtered['mode_choice'].value_counts().reindex(
    ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility'],
    fill_value=0
).reset_index()

# 4. Kolon isimlerini ayarla
mode_counts.columns = ['mode_choice', 'count']

# 5. Yüzde hesapla
total = mode_counts['count'].sum()
mode_counts['percentage'] = round((mode_counts['count'] / total) * 100, 2)

# 6. Sonuçları göster
mode_counts


,mode_choice,count,percentage
0,Service,56,7.94
1,Private Vehicle,139,19.72
2,Public Transport,140,19.86
3,Walking,369,52.34
4,Micromobility,1,0.14


<span style="font-size:22pt; color:red;"><b>Figure 151.Trip Purpose Distribution According to Disabled People (İstanbul SUMP Stage II HTS)</b></span>

#### 
Bu tablo, engelli bireylerin yaptığı yolculuklarda tercih edilen amaç gruplarının (örneğin iş, okul, sağlık, eğlence vb.) sayısal dağılımını göstermektedir.

 **Tablo Açıklaması**:
- **purpose_group**: Yolculuk amacı grubu
- **Work**: İşe gidiş
- **School**: Okula gidiş (ilköğretim, lise, üniversite)
- **Health**: Sağlık kurumuna gidiş
- **Leisure**: Sosyal ve eğlence amaçlı yolculuklar (yeme-içme, alışveriş, ziyaret, vb.)
- **Pick-up/Drop-off**: Çocuk ya da birini alma/bırakma
- **trip_count**: Her bir grup için yapılan yolculuk sayısı a da birini alma/bırakma

**Other**: Yukarıdakilere uymayan diğer amaçlar

**trip_count**: Her bir grup için yapılan yolculuk sayısı

In [78]:
import pandas as pd

# 1. Verileri birleştir (person_id üzerinden)
merged = result.merge(profile, on='person_id', how='left')

# 2. Filtreleme: Eve gidiş hariç ve disability == 'Evet'
filtered = merged[
    (merged['purpose'] != 'Eve gidiş') &
    (merged['disability'] == 'Evet')
].copy()

# 3. Amaca göre gruplama fonksiyonu
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Other'

# 4. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. Grupla ve say
grouped = (
    filtered.groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
    .sort_values('trip_count', ascending=False)
    .reset_index(drop=True)
)

# Sonuç
grouped


,purpose_group,trip_count
0,Leisure,166
1,Health,81
2,Work,52
3,School,42
4,Pick-up/Drop-off,30



<span style="font-size:22pt; color:red;"><b>Figure 154. Mode Distribution of Large Households According to Travel Purposes (İstanbul SUMP Stage II HTS)</b></span>

### 


In [79]:
import pandas as pd

# 1. Verileri birleştir
merged = result.merge(house, on='hh_id', how='left')

# 2. Filtreleme: geçerli mode_choice ve purpose olanlar, yalnız yaşamayan haneler
filtered = merged[
    merged['mode_choice'].notna() &
    merged['purpose'].notna() &
    (merged['hh_type'] != 'Yalnız yaşayan kişi')
].copy()

# 3. purpose_group dönüşümü
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    elif purpose == 'Eve gidiş':
        return 'Home'
    else:
        return 'Other'

# 4. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. Grup bazında mode_choice sayımı
grouped = (
    filtered.groupby(['purpose_group', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 6. Eksik kolonları ekle
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 7. Kolon sıralaması
grouped = grouped[[
    'purpose_group', 'Service', 'Private Vehicle',
    'Public Transport', 'Walking', 'Micromobility'
]].sort_values('purpose_group').reset_index(drop=True)

# 8. Sonuç
grouped


mode_choice,purpose_group,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Health,43,1137,1587,1299,1
1,Home,8972,18853,20168,54920,281
2,Leisure,187,3334,4069,26995,70
3,Pick-up/Drop-off,25,1097,184,6073,4
4,School,2061,1098,4890,15174,63
5,Work,7239,14605,11371,10612,170


In [80]:
# 9. Transpose: Satır ve sütunları değiştir
transposed = grouped.set_index('purpose_group').T.reset_index()
transposed = transposed.rename(columns={'index': 'mode_choice'})

# 10. Kolonlara göre yüzde hesapla
percent_df = transposed.copy()
purpose_groups = percent_df.columns[1:]  # ilk sütun mode_choice

# Her sütunun toplamına göre normalize et
percent_df[purpose_groups] = (
    percent_df[purpose_groups].div(percent_df[purpose_groups].sum(axis=0), axis=1) * 100
).round(1)

# 11. Sonuç
percent_df.to_excel("test.xlsx")


<span style="font-size:22pt; color:red;"><b>Figure 155.Mode Distribution of One Person Household According to Travel Purposes (İstanbul SUMP Stage II HTS</b></span>

#### 
Bu tablo, yalnız yaşamayan hanelerde yaşayan bireylerin farklı amaçlarla yaptıkları yolculuklarda tercih ettikleri ulaşım modlarının sayısal dağılımını göstermektedir. Yolculuk amacı (örneğin işe gidiş, okula gidiş vb.) gruplandırılarak ulaşım modu tercihlerine göre analiz edilmiştir.

 **Tablo Açıklaması**:

**purpose_group**: Yolculuğun genel amacı (örneğin: Work, School, Leisure, vb.)

**Service**: Servis aracıyla yapılan yolculukların sayısı

**Private Vehicle**: Özel araçla yapılan yolculukların sayısı

**Public Transport**: Toplu taşıma ile yapılan yolculukların sayısı

**Walking**: Yürüyerek yapılan yolculukların sayısı

**Micromobility**: Scooter, bisiklet gibi mikro mobilite araçlarıyla yapılan yolculukların sayısı


In [81]:
import pandas as pd

# 1. Verileri birleştir
merged = result.merge(house, on='hh_id', how='left')

# 2. Filtreleme: geçerli mode_choice ve purpose olanlar, yalnız yaşamayan haneler
filtered = merged[
    merged['mode_choice'].notna() &
    merged['purpose'].notna() &
    (merged['hh_type'] == 'Yalnız yaşayan kişi')
].copy()

# 3. purpose_group dönüşümü
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    elif purpose == 'Eve gidiş':
        return 'Home'
    else:
        return 'Other'

# 4. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 5. Grup bazında mode_choice sayımı
grouped = (
    filtered.groupby(['purpose_group', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 6. Eksik kolonları ekle
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 7. Kolon sıralaması
grouped = grouped[[
    'purpose_group', 'Service', 'Private Vehicle',
    'Public Transport', 'Walking', 'Micromobility'
]].sort_values('purpose_group').reset_index(drop=True)

# 8. Sonuç
grouped


mode_choice,purpose_group,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Health,4,177,209,264,5
1,Home,281,1633,1677,4464,97
2,Leisure,15,516,503,3774,52
3,Pick-up/Drop-off,0,7,2,15,0
4,School,4,66,203,50,4
5,Work,286,1047,953,737,43


In [82]:
# 9. Transpose: Satır ve sütunları değiştir
transposed = grouped.set_index('purpose_group').T.reset_index()
transposed = transposed.rename(columns={'index': 'mode_choice'})

# 10. Kolonlara göre yüzde hesapla
percent_df = transposed.copy()
purpose_groups = percent_df.columns[1:]  # ilk sütun mode_choice

# Her sütunun toplamına göre normalize et
percent_df[purpose_groups] = (
    percent_df[purpose_groups].div(percent_df[purpose_groups].sum(axis=0), axis=1) * 100
).round(1)

# 11. Sonuç
percent_df


purpose_group,mode_choice,Health,Home,Leisure,Pick-up/Drop-off,School,Work
0,Service,0.6,3.4,0.3,0.0,1.2,9.3
1,Private Vehicle,26.9,20.0,10.6,29.2,20.2,34.1
2,Public Transport,31.7,20.6,10.3,8.3,62.1,31.1
3,Walking,40.1,54.8,77.7,62.5,15.3,24.0
4,Micromobility,0.8,1.2,1.1,0.0,1.2,1.4


<span style="font-size:22pt; color:red;"><b>Figure 156.Mode Choice Based on Homeownership (İstanbul SUMP Stage II HTS)</b></span>

####  

Bu tablo, farklı hane tiplerine sahip bireylerin gerçekleştirdiği yolculuklarda tercih ettikleri ulaşım modlarının sayısal dağılımını göstermektedir.

 **Tablo Açıklaması**:

**house_type**: Hane tipi (örneğin: Çekirdek Aile, Geniş Aile, Tek Kişilik Hane vb.)

**Service**: Servis aracı ile yapılan yolculukların sayısı

**Private Vehicle**: Özel araç ile yapılan yolculukların sayısı

**Public Transport**: Toplu taşıma ile yapılan yolculukların sayısı

**Walking**: Yürüyerek yapılan yolculukların sayısı

**Micromobility**: Scooter, bisiklet gibi mikro mobilite araçları ile yapılan yolculukların sayısı

In [83]:
import pandas as pd

# 1. Verileri hh_id üzerinden birleştir
merged = result.merge(house, on='hh_id', how='left')

# 2. Geçerli mode_choice olanları filtrele
filtered = merged[
    merged['mode_choice'].notna()
].copy()

# 3. Grup bazında sayım: house_type x mode_choice
grouped = (
    filtered.groupby(['house_type', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik kolonları sıfırla
for col in ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Kolon sıralamasını yap
house_type_mode_counts = grouped[[
    'house_type',
    'Service',
    'Private Vehicle',
    'Public Transport',
    'Walking',
    'Micromobility'
]]

# 6. Sonuçları göster
house_type_mode_counts


mode_choice,house_type,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Sitede Daire,2494,9510,4185,13232,73
1,Apartman Dairesi,16071,32831,40160,107926,697
2,Diğer,24,174,57,224,0
3,Gecekondu,54,78,194,368,0
4,Müstakil Konut,474,977,1220,2627,20


In [84]:
# 7. Satır bazlı yüzdesel dağılım
house_type_mode_percent = house_type_mode_counts.copy()
mode_cols = ['Service', 'Private Vehicle', 'Public Transport', 'Walking', 'Micromobility']

house_type_mode_percent[mode_cols] = (
    house_type_mode_percent[mode_cols]
    .div(house_type_mode_percent[mode_cols].sum(axis=1), axis=0) * 100
).round(1)

# 8. Sonuçları göster
house_type_mode_percent


mode_choice,house_type,Service,Private Vehicle,Public Transport,Walking,Micromobility
0,Sitede Daire,8.5,32.2,14.2,44.9,0.2
1,Apartman Dairesi,8.1,16.6,20.3,54.6,0.4
2,Diğer,5.0,36.3,11.9,46.8,0.0
3,Gecekondu,7.8,11.2,28.0,53.0,0.0
4,Müstakil Konut,8.9,18.4,22.9,49.4,0.4


<span style="font-size:22pt; color:red;"><b>Figure 277.Purpose of Private Vehicle Trip</b></span>

Bu tablo, özel araç (Private Vehicle) ile yapılan yolculuklarda, seyahat amacına göre yapılan yolculuk sayısını ve bu sayıların toplam içindeki yüzdelik oranlarını göstermektedir.

####  **Tablo Açıklaması**:

**purpose_group**: Yolculuğun amacı (ör. Work, School, Leisure vb.)

**trip_count**: İlgili amaçla yapılan yolculuk sayısı

**percentage**: Toplam yolculuklar içindeki oranı (%)


In [85]:
import pandas as pd

# 1. 'Eve gidiş' dışı ve mode_category 'Private Vehicle' olanları filtrele
filtered = result[
    (result['purpose'] != 'Eve gidiş') &
    (result['mode_choice'] == 'Private Vehicle')
].copy()

# 2. purpose_group dönüşümü
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return None

# 3. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. Geçerli purpose_group'lara göre grupla ve say
grouped = (
    filtered[filtered['purpose_group'].notna()]
    .groupby('purpose_group')
    .size()
    .reset_index(name='trip_count')
)

# 5. Yüzdelik oranı hesapla
total = grouped['trip_count'].sum()
grouped['percentage'] = round(grouped['trip_count'] * 100 / total, 2)

# 6. Trip sayısına göre sırala
grouped = grouped.sort_values(by='trip_count', ascending=False).reset_index(drop=True)

# Sonuç:
grouped


,purpose_group,trip_count,percentage
0,Work,15652,67.80
1,Leisure,3850,16.68
2,Health,1314,5.69
3,School,1164,5.04
4,Pick-up/Drop-off,1104,4.78


<span style="font-size:22pt; color:red;"><b>Figure 278.District Based Private Vehicle Trips Purposes in İstanbul (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, özel araçla (Private Vehicle) yapılan yolculukların ilçelere (district) ve amaç gruplarına (purpose group) göre sayısal dağılımını göstermektedir.

####  **Tablo Açıklaması**:

**District**: Hanenin olduğu ilçe

**Work**: İşe gitmek amacıyla yapılan özel araç yolculuklarının sayısı

**School**: Okula gitmek amacıyla yapılan özel araç yolculuklarının sayısı

**Health**: Sağlık kurumuna gitmek amacıyla yapılan özel araç yolculuklarının sayısı

**Leisure**: Eğlence, yeme-içme, alışveriş, ziyaret gibi sosyal amaçlarla yapılan özel araç yolculuklarının sayısı

**Pick-up/Drop-off**: Çocuk ya da başka birini alma/bırakma amaçlı özel araç yolculuklarının sayısı 


In [86]:
import pandas as pd

# 1. Geçerli amaç ve Private Vehicle olanları filtrele
filtered = result[
    (result['purpose'].notna()) &
    (result['purpose'] != 'Eve gidiş') &
    (result['mode_category'] == 'Private Vehicle')
].copy()

# 2. Purpose grup dönüşümü
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return None

# 3. Yeni kolon oluştur
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. Geçerli purpose_group olanları filtrele
filtered = filtered[filtered['purpose_group'].notna()]

# 5. Grup bazında sayım: district x purpose_group
grouped = (
    filtered.groupby(['district', 'purpose_group'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 6. Sonuçları göster
grouped.head()


purpose_group,district,Health,Leisure,Pick-up/Drop-off,School,Work
0,ADALAR,0,1,0,0,5
1,ARNAVUTKÖY,23,43,11,5,207
2,ATAŞEHİR,22,68,17,35,414
3,AVCILAR,27,85,19,35,336
4,BAHÇELİEVLER,68,98,54,71,719


<span style="font-size:22pt; color:red;"><b>Figure 281.Distribution of Private Vehicle Trips According to 24-Hour Purposes (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, özel araç (Private Vehicle) kullanılarak yapılan yolculukların, günün saatlerine göre ve yolculuk amacına (iş, okul, sağlık, vb.) göre nasıl dağıldığını göstermektedir.

####  **Tablo Açıklaması**:

**hour_group**: Yolculuğun başladığı saat (örneğin: 08:00)

**Work**: İşe gidiş amaçlı yapılan yolculuk sayısı

**School**: Okula gidiş amaçlı yapılan yolculuk sayısı

**Health**: Sağlık kurumuna gidiş amaçlı yapılan yolculuk sayısı

**Leisure**: Eğlence, yeme-içme, alışveriş, ziyaret gibi amaçlarla yapılan yolculuk sayısı

**Pick-up/Drop-off**: Biri bırakma ya da alma amaçlı yapılan yolculuk sayısı


In [87]:
import pandas as pd

# 1. Gerekli dönüşümler ve filtreler
filtered = result[
    (result['mode_choice'] == 'Private Vehicle') &
    result['purpose'].notna() &
    (result['purpose'] != 'Eve gidiş') &
    result['start_time'].notna()
].copy()

# 2. Saat dilimini çıkart (örn: "08:00")
filtered['hour_group'] = pd.to_datetime(filtered['start_time']).dt.strftime('%H:00')

# 3. purpose_group haritalama
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)', 'Okula gidiş(lise)', 'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş', 'Yeme/içme Gidiş', 'Alışveriş',
        'Akraba/Arkadaş  Ziyareti', 'İş takibi/ Ödemeler', 'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return 'Unknown'

filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. Gruplama: hour_group x purpose_group
grouped = (
    filtered.groupby(['hour_group', 'purpose_group'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .sort_values('hour_group')
)

# 5. Sonuç
grouped.head()


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_5020\1565479256.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered['hour_group'] = pd.to_datetime(filtered['start_time']).dt.strftime('%H:00')


purpose_group,hour_group,Health,Leisure,Pick-up/Drop-off,School,Work
0,00:00,0,0,0,0,10
1,01:00,6,11,0,0,4
2,02:00,1,1,0,0,6
3,03:00,1,0,0,0,17
4,04:00,0,0,2,0,42


<span style="font-size:22pt; color:red;"><b>Figure 403. Daily Temporal Pedestrian Movement Pattern (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, yürüyerek (Walking) yapılan yolculukların gün içindeki saatlik zaman dilimlerine göre sayısal dağılımını göstermektedir. Yolculukların başladığı saat dikkate alınarak her bir saat aralığı için toplam yürüyüş yolculuğu sayısı verilmiştir.

#### **Tablo Açıklaması**:
**hour_group**: Yolculuğun başladığı saat dilimi (örneğin: 08:00)

**trip_count**: O saat diliminde yapılan yürüyüş yolculuklarının sayısı

In [88]:
# 1. Yalnızca geçerli mode_choice ve purpose olanlar, mode_choice = 'Walking' filtrele
filtered = result[
    (result['mode_choice'].notna()) &
    (result['purpose'].notna()) &
    (result['mode_choice'] == 'Walking')
].copy()

# 2. Saat grubunu oluştur (örn. 08:00 gibi)
filtered['hour_group'] = pd.to_datetime(filtered['start_time']).dt.strftime('%H:00')

# 3. Grup bazında sayım
walking_hourly_counts = (
    filtered.groupby('hour_group')
    .size()
    .reset_index(name='trip_count')
    .sort_values('hour_group')
)
walking_hourly_counts


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_5020\3382900910.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered['hour_group'] = pd.to_datetime(filtered['start_time']).dt.strftime('%H:00')


,hour_group,trip_count
0,00:00,12
1,01:00,57
2,02:00,45
3,03:00,18
4,04:00,35
5,05:00,117
6,06:00,882
7,07:00,11139
8,08:00,13465
9,09:00,4113


<span style="font-size:22pt; color:red;"><b>Figure 405. Age Distribution of Walking and Micromobility Individuals (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, bireylerin yaş gruplarına göre yürüyüş ve mikromobilite (ör. scooter, bisiklet) modlarını ne sıklıkla tercih ettiğini göstermektedir.

####  **Tablo Açıklaması**:
**age_group**: Katılımcının yaş aralığı

**Walking**: Yürüme moduyla yapılan yolculuk sayısı

**Micromobility**: Mikromobilite (scooter, bisiklet vb.) modlarıyla yapılan yolculuk sayısı

In [89]:
import pandas as pd

# 1. Trip ve profile verilerini birleştir (person_id üzerinden)
merged = result.merge(profile, on='person_id', how='left')

# 2. Geçerli yaş ve yürüme/mikromobilite içeren verileri filtrele
filtered = merged[
    merged['mode_choice'].isin(['Walking', 'Micromobility']) &
    merged['mode_choice'].notna() &
    merged['age'].notna()
].copy()

# 3. Yaş grubu oluştur
def get_age_group(age):
    if 0 <= age <= 5:
        return '0-5'
    elif 6 <= age <= 9:
        return '6-9'
    elif 10 <= age <= 14:
        return '10-14'
    elif 15 <= age <= 19:
        return '15-19'
    elif 20 <= age <= 24:
        return '20-24'
    elif 25 <= age <= 29:
        return '25-29'
    elif 30 <= age <= 34:
        return '30-34'
    elif 35 <= age <= 39:
        return '35-39'
    elif 40 <= age <= 44:
        return '40-44'
    elif 45 <= age <= 49:
        return '45-49'
    elif 50 <= age <= 54:
        return '50-54'
    elif 55 <= age <= 59:
        return '55-59'
    elif 60 <= age <= 64:
        return '60-64'
    elif age >= 65:
        return '65+'
    else:
        return 'Unknown'

filtered['age_group'] = filtered['age'].apply(get_age_group)

# 4. Yaş grubu ve mode_category'ye göre sayım yap
grouped = (
    filtered.groupby(['age_group', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 5. Eksik kolonları sıfırla
for col in ['Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 6. Kolonları sıralı hale getir
grouped = grouped[['age_group', 'Walking', 'Micromobility']].sort_values('age_group').reset_index(drop=True)

# 7. Sonuçları göster
grouped


mode_choice,age_group,Walking,Micromobility
0,0-5,4333,1
1,10-14,15592,47
2,15-19,8114,80
3,20-24,5708,80
4,25-29,10175,193
5,30-34,10717,95
6,35-39,9115,79
7,40-44,8260,50
8,45-49,6451,25
9,50-54,6447,16


<span style="font-size:22pt; color:red;"><b>Figure 406. Gender Distribution of Walking and Micromobility Individuals (İstanbul SUMP Stage II HTS)</b></span>



####  **Tablo Açıklaması**:


In [90]:
filtered_result = result[result["mode_choice"].isin(["Micromobility", "Walking"])]
gender_mode_counts = (
    filtered_result
    .groupby(["mode_choice", "gender"])
    .size()
    .unstack(fill_value=0)
)
gender_mode_counts

gender,Belirtmek istemiyor,Erkek,Kadın
mode_choice,,,
Micromobility,0,637,153
Walking,663,58590,65124


<span style="font-size:22pt; color:red;"><b>Figure 409. Driver's License Ownership Distribution of Active Mobility Individuals (İstanbul SUMP Stage II HTS</b></span>

Bu tablo, Walking (Yürüyüş) ve Micromobility (scooter, bisiklet vb.) modlarını tercih eden bireylerin, sürücü belgesi sahibi olup olmadıklarına göre dağılımını göstermektedir.

#### **Tablo Açıklaması**:

**mode_choice**: Tercih edilen ulaşım türü

**drive_license_status**:

**Evet**: Sürücü belgesi var

**Hayır**: Sürücü belgesi yok

**Bilinmiyor**: Veride eksiklik veya bilinmeyen değer

**person_count**: İlgili gruba ait birey sayısı

In [91]:
import pandas as pd

# 1. Verileri birleştir (kişisel bilgilerle)
merged = result.merge(profile, on='person_id', how='left')

# 2. Geçerli koşullara göre filtrele (mode_choice ve drive_lic en azından boş olmayanlar)
filtered = merged[
    merged['mode_choice'].isin(['Walking', 'Micromobility'])
].copy()

# 3. Sürücü belgesi durumunu normalize et
def map_drive_license(val):
    if pd.isna(val):
        return 'Eksik'  # NaN değerleri de etiketle
    val = str(val).strip().lower()
    if val == 'var':
        return 'Evet'
    elif val == 'yok':
        return 'Hayır'
    else:
        return 'Bilinmiyor'

filtered['drive_license_status'] = filtered['drive_lic'].apply(map_drive_license)

# 4. Grup bazlı sayım (Eksik, Bilinmiyor gibi gruplar dahil)
grouped = (
    filtered.groupby(['mode_choice', 'drive_license_status'], dropna=False)
    .size()
    .reset_index(name='person_count')
    .sort_values(['mode_choice', 'drive_license_status'])
)

grouped

,mode_choice,drive_license_status,person_count
0,Micromobility,Eksik,84
1,Micromobility,Evet,488
2,Micromobility,Hayır,218
3,Walking,Eksik,32652
4,Walking,Evet,43639
5,Walking,Hayır,48086


<span style="font-size:22pt; color:red;"><b>Figure 410. Income Status Distribution of Walking and Micromobility Individuals (İstanbul SUMP Stage II HTS)</b></span>

Bu tablo, bireylerin hane gelir düzeylerine göre hangi aktif ulaşım türlerini (yürüme ve mikromobilite) ne sıklıkla kullandığını göstermektedir.


####  **Tablo Açıklaması**:

**Gelir Grubu**: Hanelerin aylık gelir düzeylerine göre gruplandırılması.

**Walking**: İlgili gelir grubundaki bireylerin yürüyerek gerçekleştirdiği yolculuk sayısı.

**Micromobility**: Scooter, bisiklet gibi mikro mobilite araçlarıyla yapılan yolculuk sayısı.

In [92]:
import pandas as pd

# 1. result ve house veri setlerini hh_id üzerinden birleştir
merged = result.merge(house[['hh_id', 'hh_income']], on='hh_id', how='left')

# 2. İlgili modlara ve geçerli gelir bilgisine sahip verileri filtrele
filtered = merged[
    (merged['mode_choice'].isin(['Walking', 'Micromobility'])) &
    (merged['hh_income'].notna())
].copy()

# 3. Gelir grubunu sınıflandır
def classify_income(income):
    income = income.strip()
    if income in ['5.000 TL’den az', '5.001 – 10.000 TL', '10.001 - 15.000 TL', '15.001 - 25.000 TL']:
        return '25.000 TL altı'
    elif income in ['25.001 - 40.000 TL', '40.001 - 60.000 TL', '60.001 - 80.000 TL']:
        return '25.001 - 80.000 TL'
    elif income in [
        '100.000TL’den fazla', '60.001 - 100.000 TL', '80.001 - 100.000 TL',
        '100.001 - 140.000 TL', '140.001 - 200.000 TL', '200.001TL’den fazla'
    ]:
        return '80.001 TL üstü'
    else:
        return 'Bilinmiyor'

filtered['income_group'] = filtered['hh_income'].apply(classify_income)

# 4. Grup bazlı sayım
grouped = (
    filtered.groupby(['income_group', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 5. Eksik kolonları sıfırla
for col in ['Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 6. Sütun sıralaması
grouped = grouped[['income_group', 'Walking', 'Micromobility']].sort_values('income_group').reset_index(drop=True)

grouped

mode_choice,income_group,Walking,Micromobility
0,25.000 TL altı,43703,230
1,25.001 - 80.000 TL,52962,403
2,80.001 TL üstü,4610,15
3,Bilinmiyor,23102,142


<span style="font-size:22pt; color:red;"><b>Figure 411. Public Transportation Card Ownership Distribution of Walking and Micromobility Individuals</b></span>


Bu tablo, bireylerin toplu taşıma kartı (pt_card) sahiplik durumuna göre yürüme (Walking) ve mikromobilite (Micromobility) ulaşım modlarını kullanma sayılarını göstermektedir.

####  **Tablo Açıklaması**:

**mode_category**: Tercih edilen ulaşım modu (örneğin: Walking, Micromobility)

**pt_card_status**: Toplu taşıma kartı durumu (Evet, Hayır, Bilinmiyor)

**person_count**: İlgili grupta yer alan kişi sayısı

In [93]:
import pandas as pd

# 1. Verileri birleştir
merged = result.merge(profile, on='person_id', how='left')

# 2. Yürüyüş ve mikromobilite modlarını filtrele, pt_card boş olmayanlar
filtered = merged[
    merged['mode_choice'].isin(['Walking', 'Micromobility']) &
    merged['mode_choice'].notna() &
    merged['pt_card'].notna()
].copy()

# 3. Kart durumu kategorisi oluştur
def map_pt_card(card):
    card = str(card).strip().lower()
    if card == 'var':
        return 'Evet'
    elif card == 'yok':
        return 'Hayır'
    else:
        return 'Bilinmiyor'

filtered['pt_card_status'] = filtered['pt_card'].apply(map_pt_card)

# 4. Grup bazlı sayım: mode_category x pt_card_status
grouped = (
    filtered.groupby(['mode_choice', 'pt_card_status'])
    .size()
    .reset_index(name='person_count')
    .sort_values(['mode_choice', 'pt_card_status'])
)
grouped



,mode_choice,pt_card_status,person_count
0,Micromobility,Evet,712
1,Micromobility,Hayır,77
2,Walking,Evet,91786
3,Walking,Hayır,28258


<span style="font-size:22pt; color:red;"><b>Figure 412. Student Individual Distribution of Walking and Micromobility Individuals</b></span>

#### 
Bu tablo, bireylerin eğitim düzeylerine göre yürüyüş (Walking) ve mikromobilite (Micromobility) (örn. scooter, bisiklet) tercihlerini göstermektedir.

In [94]:
import pandas as pd

# 1. result ve profile tablolarını birleştir
merged = result.merge(profile[['person_id', 'edu']], on='person_id', how='left')

# 2. Geçerli mode_category ve education bilgisi olan, istenilen modları filtrele
filtered = merged[
    (merged['mode_choice'].isin(['Walking', 'Micromobility'])) &
    (merged['edu'].notna())
].copy()

# 3. Grup bazında sayım: education_level x mode_category
grouped = (
    filtered.groupby(['edu', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={'edu': 'education_level'})
)

# 4. Eksik kolonları sıfırla
for col in ['Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Kolon sıralamasını yap
education_mode_counts = grouped[['education_level', 'Walking', 'Micromobility']]

# 6. Sonuçları göster
education_mode_counts


mode_choice,education_level,Walking,Micromobility
0,Açıköğretim lise öğrenci,385,3
1,Açıköğretim ortaöğretim öğrenci,154,4
2,Açıköğretim üniversite öğrenci,298,4
3,Kreş/Okul öncesi öğrenci,753,1
4,Lise/ Dengi öğrenci,6312,70
5,Lise/Dengi mezun,24090,279
6,Okula başlamamış,4281,3
7,Okur-Yazar Değil,1782,1
8,Okur-Yazar Fakat Bir Okul Bitirmemiş,1862,0
9,Ortaöğretim mezun,17205,105


<span style="font-size:22pt; color:red;"><b>Figure 413. Employment Status Distribution of Walking and Micromobility Individuals</b></span>

Bu tablo, bireylerin çalışma durumuna (örneğin çalışan, işsiz, öğrenci vb.) göre tercih ettikleri yürüme (Walking) ve mikromobilite (Micromobility) ulaşım modlarının sayısal dağılımını göstermektedir.

####  **Tablo Açıklaması**:

**work_status**: Bireyin çalışma durumu (örn. Çalışıyor, İşsiz)

**Walking**: Yürüyerek yapılan yolculuk sayısı

**Micromobility**: Scooter, bisiklet gibi mikromobilite araçlarıyla yapılan yolculuk sayısı

In [95]:
import pandas as pd

# 1. Trip ve profile verilerini person_id üzerinden birleştir
merged = result.merge(profile[['person_id', 'work_status']], on='person_id', how='left')

# 2. Filtreleme: sadece Walking ve Micromobility, work_status boş olmayanlar
filtered = merged[
    merged['mode_choice'].isin(['Walking', 'Micromobility']) &
    merged['work_status'].notna()
].copy()

# 3. Grup bazında sayım: work_status x mode_category
grouped = (
    filtered.groupby(['work_status', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 4. Eksik mod kolonlarını sıfırla (varsa)
for col in ['Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 5. Sonuçları sırala
grouped = grouped.sort_values(by='work_status').reset_index(drop=True)

# 6. Sonucu göster
grouped.head()


mode_choice,work_status,Micromobility,Walking
0,Çalışmıyor,312,92493
1,Çalışıyor,477,27551


<span style="font-size:22pt; color:red;"><b>Figure 414. Trip Purpose Distribution of Walking and Micromobility Individuals</b></span>

Bu tablo, yolculuk amacı gruplarına (purpose_group) göre, bireylerin yürüme (Walking) ve mikromobilite (Micromobility) (örneğin scooter, bisiklet) modlarını ne sıklıkla kullandığını göstermektedir. Tablo, yalnızca bu iki ulaşım modunu içeren yolculuklar üzerinden oluşturulmuştur ve eve dönüş yolculukları hariç tutulmuştur.

#### **Tablo Açıklaması**:

**Work**: İşe gitmek amacıyla yapılan yolculuklar

**School**: Okula gitmek (ilköğretim, lise, üniversite)

**Health**: Sağlık kuruluşlarına yapılan yolculuklar

**Leisure**: Sosyal ve eğlence amaçlı yolculuklar (alışveriş, ziyaret, yeme-içme vb.)

**Pick-up/Drop-off**: Çocuk veya diğer kişilerin alınması/bırakılması

In [96]:
import pandas as pd

# 1. mode_category ve first_purpose geçerli olan ve eve gidiş olmayanları filtrele
filtered = result[
    result['mode_choice'].isin(['Walking', 'Micromobility']) &
    result['mode_choice'].notna() &
    result['purpose'].notna() &
    (result['purpose'] != 'Eve gidiş')
].copy()

# 2. first_purpose'ı purpose_group'a dönüştür
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in ['Çocuk Alma/Bırakma', 'Diğer Alma/Bırakma']:
        return 'Pick-up/Drop-off'
    else:
        return None

filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 3. Geçerli purpose_group olanları al
filtered = filtered[filtered['purpose_group'].notna()]

# 4. Grup bazında sayım: purpose_group x mode_category
grouped = (
    filtered.groupby(['purpose_group', 'mode_choice'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 5. Eksik mod kolonlarını sıfırla
for col in ['Walking', 'Micromobility']:
    if col not in grouped.columns:
        grouped[col] = 0

# 6. Sonuçları göster
grouped = grouped[['purpose_group', 'Walking', 'Micromobility']].sort_values('purpose_group').reset_index(drop=True)


grouped

mode_choice,purpose_group,Walking,Micromobility
0,Health,1563,6
1,Leisure,30769,122
2,Pick-up/Drop-off,6088,4
3,School,15224,67
4,Work,11349,213


<span style="font-size:22pt; color:red;"><b>Figure 278. Purpose of Private Vehicle Trips (İstanbul SUMP Stage II HTS)</b></span>



In [100]:
# 1. "Eve gidiş" dışındaki ve "Private Vehicle" olmayan satırları filtrele
filtered = result[
    (result['purpose'] != 'Eve gidiş') & 
    (result['mode_choice'] == 'Private Vehicle')
].copy()

# 2. first_purpose değerlerini gruplayarak yeni sütun oluştur
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 3. Yeni kolon ekle
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. NULL olmayanlarla çalış ve grup bazında say
grouped = filtered[filtered['purpose_group'].notna()] \
    .groupby('purpose_group') \
    .size() \
    .reset_index(name='trip_count')

# 5. Yüzde hesapla
total_trips = grouped['trip_count'].sum()
grouped['percentage'] = round(grouped['trip_count'] * 100.0 / total_trips, 2)

# 6. Azalan sıraya göre sırala
grouped = grouped.sort_values(by='trip_count', ascending=False).reset_index(drop=True)
grouped

,purpose_group,trip_count,percentage
0,Work,15652,67.80
1,Leisure,3850,16.68
2,Health,1314,5.69
3,School,1164,5.04
4,Pick-up/Drop-off,1104,4.78


<span style="font-size:22pt; color:red;"><b>Figure 279-280-281. District Based Private Vehicle Trips Purposes in İstanbul (İstanbul SUMP Stage II HTS)</b></span>



In [131]:
# 1. "Eve gidiş" dışındaki ve sadece Private Vehicle olan satırları filtrele
filtered = result[
    (result['purpose'] != 'Eve gidiş') &
    (result['mode_choice'] == 'Private Vehicle')
].copy()

# 2. Amaçları gruplandıran fonksiyon
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 3. Yeni kolon: purpose_group
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. NULL olmayanlar üzerinden district ve purpose_group kırılımı
grouped = filtered[filtered['purpose_group'].notna()] \
    .groupby(['district', 'purpose_group']) \
    .size() \
    .reset_index(name='trip_count')

# 5. District bazlı yüzde hesapla
grouped['percentage'] = grouped.groupby('district')['trip_count'] \
    .transform(lambda x: round(x * 100 / x.sum(), 2))

# 6. Gerekirse sıralama (isteğe bağlı)
grouped = grouped.sort_values(by=['district', 'trip_count'], ascending=[True, False]) \
    .reset_index(drop=True)

# Pivot tablo: purpose_group değerlerini ayrı sütun yap ve yüzdeyi göster
pivot_df = grouped.pivot_table(
    index='district',
    columns='purpose_group',
    values='trip_count',
    fill_value=0
).reset_index()

# Kolonları belirli sıraya göre yeniden sırala (isteğe bağlı)
desired_order = ['Work', 'School', 'Health', 'Leisure', 'Pick-up/Drop-off']
pivot_df = pivot_df[['district'] + [col for col in desired_order if col in pivot_df.columns]]
pivot_df

purpose_group,district,Work,School,Health,Leisure,Pick-up/Drop-off
0,ADALAR,5,0,0,1,0
1,ARNAVUTKÖY,207,5,23,43,11
2,ATAŞEHİR,414,35,22,68,17
3,AVCILAR,334,33,27,85,19
4,BAHÇELİEVLER,717,71,68,98,54
5,BAKIRKÖY,217,18,29,79,24
6,BAYRAMPAŞA,386,10,48,55,38
7,BAĞCILAR,649,65,53,74,20
8,BAŞAKŞEHİR,676,62,33,116,79
9,BEYKOZ,99,2,25,39,0


<span style="font-size:22pt; color:red;"><b>Figure 282. Distribution of Private Vehicle Trips According to 24-Hour Purposes (İstanbul SUMP Stage II HTS)</b></span>

In [139]:
import pandas as pd

# 1. "Eve gidiş" dışındaki ve sadece Private Vehicle olan satırları filtrele
filtered = result[
    (result['purpose'] != 'Eve gidiş') &
    (result['mode_choice'] == 'Private Vehicle')
].copy()

# 2. Amaçları gruplandıran fonksiyon
def map_purpose(purpose):
    if purpose == 'İşe gidiş':
        return 'Work'
    elif purpose in [
        'Okula gidiş(ilk ve ortaöğretim)',
        'Okula gidiş(lise)',
        'Okula gidiş(üniversite ve üzeri)'
    ]:
        return 'School'
    elif purpose == 'Sağlık Kurumuna gidiş':
        return 'Health'
    elif purpose in [
        'Spor, Eğlence vb. gidiş',
        'Yeme/içme Gidiş',
        'Alışveriş',
        'Akraba/Arkadaş  Ziyareti',
        'İş takibi/ Ödemeler',
        'Diğer'
    ]:
        return 'Leisure'
    elif purpose in [
        'Çocuk Alma/Bırakma',
        'Diğer Alma/Bırakma'
    ]:
        return 'Pick-up/Drop-off'
    else:
        return None

# 3. purpose_group kolonu
filtered['purpose_group'] = filtered['purpose'].apply(map_purpose)

# 4. Saatlik zaman aralığına göre start_hour formatını oluştur
filtered['start_hour'] = pd.to_datetime(filtered['start_time']).dt.hour
filtered['start_hour'] = filtered['start_hour'].apply(lambda x: f"{x:02d}:00 - {((x + 1)%24):02d}:00")

# 5. NULL olmayanlar üzerinden start_hour ve purpose_group kırılımı
grouped = filtered[filtered['purpose_group'].notna()] \
    .groupby(['start_hour', 'purpose_group']) \
    .size() \
    .reset_index(name='trip_count')

# 6. Pivot tablo: saatlik aralıklar satır, purpose_group sütun olacak şekilde
pivot_df = grouped.pivot_table(
    index='start_hour',
    columns='purpose_group',
    values='trip_count',
    fill_value=0
).reset_index()

# 7. Saatlik sıralama için start_hour sıralı hale getir
hour_order = [f"{h:02d}:00 - {(h+1)%24:02d}:00" for h in range(24)]
pivot_df['start_hour'] = pd.Categorical(pivot_df['start_hour'], categories=hour_order, ordered=True)
pivot_df = pivot_df.sort_values('start_hour').reset_index(drop=True)

# 8. Kolon sıralaması (isteğe bağlı)
desired_order = ['start_hour', 'Work', 'School', 'Health', 'Leisure', 'Pick-up/Drop-off']
pivot_df = pivot_df[[col for col in desired_order if col in pivot_df.columns]]

# 9. En alta toplam satırı ekle
totals = pivot_df.drop(columns='start_hour').sum().to_dict()
totals['start_hour'] = 'Toplam'
pivot_df = pd.concat([pivot_df, pd.DataFrame([totals])], ignore_index=True)

pivot_df


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_5020\3420036195.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered['start_hour'] = pd.to_datetime(filtered['start_time']).dt.hour


,start_hour,Work,School,Health,Leisure,Pick-up/Drop-off
0,00:00 - 01:00,10,0,0,0,0
1,01:00 - 02:00,4,0,6,11,0
2,02:00 - 03:00,6,0,1,1,0
3,03:00 - 04:00,17,0,1,0,0
4,04:00 - 05:00,42,0,0,0,2
5,05:00 - 06:00,167,0,1,4,2
6,06:00 - 07:00,1183,14,8,18,17
7,07:00 - 08:00,5294,283,61,54,197
8,08:00 - 09:00,5995,517,174,122,402
9,09:00 - 10:00,1652,109,247,180,64


<span style="font-size:22pt; color:red;"><b>Figure 283. Distances of Private Vehicle Trips in İstanbul (İstanbul SUMP Stage II HTS)</b></span>

In [129]:
from geopy.distance import geodesic

# Sadece özel araçla yapılan yolculukları filtrele
private_vehicle_df = result[result['mode_choice'] == 'Private Vehicle'].copy()

# Geçersiz koordinatları temizle ("0.00000000° N" veya "0.00000000° E" olanları çıkar)
invalid_coords = ['0.00000000° N', '0.00000000° E']
private_vehicle_df = private_vehicle_df[
    ~private_vehicle_df['orig_lat'].isin(invalid_coords) &
    ~private_vehicle_df['orig_lon'].isin(invalid_coords) &
    ~private_vehicle_df['dest_lat'].isin(invalid_coords) &
    ~private_vehicle_df['dest_lon'].isin(invalid_coords)
]

# 1. Koordinatları temizleyen fonksiyon (derece işareti ve yönleri temizler)
def clean_coordinate(coord_str):
    if isinstance(coord_str, str):
        coord_str = coord_str.replace('°', '').replace('N', '').replace('E', '').strip()
    try:
        return float(coord_str)
    except:
        return None

# 2. Tüm koordinatları float'a çevir
private_vehicle_df['orig_lat_clean'] = private_vehicle_df['orig_lat'].apply(clean_coordinate)
private_vehicle_df['orig_lon_clean'] = private_vehicle_df['orig_lon'].apply(clean_coordinate)
private_vehicle_df['dest_lat_clean'] = private_vehicle_df['dest_lat'].apply(clean_coordinate)
private_vehicle_df['dest_lon_clean'] = private_vehicle_df['dest_lon'].apply(clean_coordinate)

# 3. Kuş uçuşu mesafe hesaplama (km)
def calculate_distance_km(row):
    if None in [
        row['orig_lat_clean'], row['orig_lon_clean'],
        row['dest_lat_clean'], row['dest_lon_clean']
    ]:
        return None
    orig = (row['orig_lat_clean'], row['orig_lon_clean'])
    dest = (row['dest_lat_clean'], row['dest_lon_clean'])
    return geodesic(orig, dest).km

# 4. Mesafeyi hesapla
private_vehicle_df['distance_km'] = private_vehicle_df.apply(calculate_distance_km, axis=1)

# 5. Boş ve 0 mesafeli satırları çıkar
private_vehicle_df = private_vehicle_df[
    private_vehicle_df['distance_km'].notnull() 
]


district_avg_distance = private_vehicle_df.groupby('district')['distance_km'].mean().reset_index()
district_avg_distance.head(2)

,district,distance_km
0,ADALAR,0.411121
1,ARNAVUTKÖY,6.389833


In [140]:
from geopy.distance import geodesic
import pandas as pd

# 1. Sadece özel araçla yapılan yolculukları filtrele
private_vehicle_df = result[result['mode_choice'] == 'Private Vehicle'].copy()

# 2. Geçersiz koordinatları temizle ("0.00000000° N" veya "0.00000000° E" olanları çıkar)
invalid_coords = ['0.00000000° N', '0.00000000° E']
private_vehicle_df = private_vehicle_df[
    ~private_vehicle_df['orig_lat'].isin(invalid_coords) &
    ~private_vehicle_df['orig_lon'].isin(invalid_coords) &
    ~private_vehicle_df['dest_lat'].isin(invalid_coords) &
    ~private_vehicle_df['dest_lon'].isin(invalid_coords)
]

# 3. Koordinatları temizleyen fonksiyon (derece işareti ve yönleri temizler)
def clean_coordinate(coord_str):
    if isinstance(coord_str, str):
        coord_str = coord_str.replace('°', '').replace('N', '').replace('E', '').strip()
    try:
        return float(coord_str)
    except:
        return None

# 4. Tüm koordinatları float'a çevir
private_vehicle_df['orig_lat_clean'] = private_vehicle_df['orig_lat'].apply(clean_coordinate)
private_vehicle_df['orig_lon_clean'] = private_vehicle_df['orig_lon'].apply(clean_coordinate)
private_vehicle_df['dest_lat_clean'] = private_vehicle_df['dest_lat'].apply(clean_coordinate)
private_vehicle_df['dest_lon_clean'] = private_vehicle_df['dest_lon'].apply(clean_coordinate)

# 5. Kuş uçuşu mesafe hesaplama (km)
def calculate_distance_km(row):
    if None in [
        row['orig_lat_clean'], row['orig_lon_clean'],
        row['dest_lat_clean'], row['dest_lon_clean']
    ]:
        return None
    orig = (row['orig_lat_clean'], row['orig_lon_clean'])
    dest = (row['dest_lat_clean'], row['dest_lon_clean'])
    return geodesic(orig, dest).km

# 6. Mesafeyi hesapla
private_vehicle_df['distance_km'] = private_vehicle_df.apply(calculate_distance_km, axis=1)

# 7. Boş ve 0 mesafeli satırları çıkar
private_vehicle_df = private_vehicle_df[
    private_vehicle_df['distance_km'].notnull() &
    (private_vehicle_df['distance_km'] > 0)
]

# 8. İstanbul ilçeleri listesi
istanbul_districts = [
    'ADALAR', 'ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER', 'BAKIRKÖY',
    'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ', 'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE',
    'ÇATALCA', 'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜPSULTAN', 'FATİH', 'GAZİOSMANPAŞA',
    'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK',
    'SANCAKTEPE', 'SARIYER', 'ŞİLE', 'SİLİVRİ', 'ŞİŞLİ', 'SULTANBEYLİ', 'SULTANGAZİ',
    'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU'
]

# 9. Sadece İstanbul ilçeleri listesinde olan satırları filtrele
private_vehicle_df_filtered = private_vehicle_df[private_vehicle_df['district'].isin(istanbul_districts)]

# 10. İlçe bazında ortalama mesafe hesapla
district_avg_distance = private_vehicle_df_filtered.groupby('district')['distance_km'].mean().reset_index()

# 11. Genel ortalama mesafeyi hesapla
overall_avg = private_vehicle_df_filtered['distance_km'].mean()

# 12. Genel ortalama satırını oluştur
overall_avg_row = pd.DataFrame({'district': ['GENEL ORTALAMA'], 'distance_km': [overall_avg]})

# 13. Sonuçları birleştir
district_avg_distance_with_overall = pd.concat([district_avg_distance, overall_avg_row], ignore_index=True)

district_avg_distance_with_overall


,district,distance_km
0,ADALAR,0.411121
1,ARNAVUTKÖY,6.500002
2,ATAŞEHİR,9.139383
3,AVCILAR,7.350030
4,BAHÇELİEVLER,5.962621
5,BAKIRKÖY,6.179111
6,BAYRAMPAŞA,6.035692
7,BAĞCILAR,6.610541
8,BAŞAKŞEHİR,8.612342
9,BEYKOZ,9.760303
